<a href="https://colab.research.google.com/github/joekelly211/masfi/blob/main/5_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports, directories and global functions

In [ ]:
# Define base directory
# Use '/content/drive/MyDrive/' for a personal drive
# Use '/gdrive/Shareddrives/' for a shared drive (must be created first)

base_dir = "/gdrive/Shareddrives/masfi"
# base_dir = '/content/drive/MyDrive/masfi'

# Mount Google Drive
from google.colab import drive
import os
import sys
if base_dir.startswith('/gdrive/Shareddrives/'):
  drive.mount('/gdrive', force_remount=True)
elif base_dir.startswith('/content/drive/MyDrive/'):
  drive.mount('/content/drive', force_remount=True)
  os.makedirs(base_dir, exist_ok=True)
else: print("Create a base_dir beginning with '/gdrive/Shareddrives/' or '/content/drive/MyDrive/'.")

_path_to_add = os.path.realpath(base_dir)
if _path_to_add not in sys.path:
    sys.path.append(_path_to_add)

In [ ]:
# Capture outputs
%%capture
# Imports and upgrades
!pip install geopandas
!pip install shap
!pip install tabulate
!pip install xgboost

In [ ]:
# Imports
import ast
from datetime import datetime, timedelta, timezone
from google.colab import runtime
import ipywidgets as widgets
import json
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from os import makedirs
from os.path import exists, join
import pandas as pd
from pathlib import Path
import pickle
import random
import re
from scipy.stats import randint, uniform
from scipy.stats.qmc import LatinHypercube
import shap
from sklearn.metrics import (
    root_mean_squared_error,
    r2_score,
    accuracy_score,
    log_loss,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
    confusion_matrix,
    matthews_corrcoef,
    balanced_accuracy_score,
    average_precision_score,
    cohen_kappa_score,
    brier_score_loss,
)
from sklearn.model_selection import KFold, StratifiedKFold
from tabulate import tabulate
import warnings
import xgboost as xgb

In [ ]:
# 4_datasets directories
datasets_dir = join(base_dir, "4_datasets/final")

# 5_models directories
models_dir = join(base_dir, "5_models")

# Create directories
makedirs(models_dir, exist_ok=True)

# Compile new model dataset (optional)

## Select a dataset to import

In [ ]:
# Select a final dataset compiled with features in 4_datasets.ipynb
for file in os.listdir(datasets_dir):
  if file.endswith(".pkl"):
    print(f'selected_dataset = "{file}"')

In [ ]:
selected_dataset = "agbd.pkl"

# Load dataset
print(f"Importing '{selected_dataset}'")
dataset_read = pd.read_pickle(join(datasets_dir, selected_dataset))
dataset_imported = dataset_read.copy().reset_index(drop=True)

# Drop geometry to save memory
# Can be added back later for accuracy testing using the ID column
dataset_imported = dataset_imported.drop(columns=['tar_geometry'])

# Print dataset summary
print(f"'{selected_dataset}' with {len(dataset_imported)} rows and {len(dataset_imported.columns)} columns imported.")

## Select ID column

In [ ]:
# Check columns and select ID column
for col in dataset_imported.columns:
  if col[:4] == "tar_":
    print(f'id_column = "{col}"')

In [ ]:
id_column = "tar_shot_number"

dataset_col_list = [id_column]

## Select a target to predict

In [ ]:
for col in dataset_imported.columns:
  if col[:4] == "tar_" and col not in dataset_col_list:
    print(f'selected_target = "{col}"')

In [ ]:
selected_target = "tar_agbd"

dataset_col_list = dataset_col_list + [selected_target]

## Select uncertainty metrics

In [ ]:
# The uncertainty metrics should allow estimation of one standard deviation of the selected target.

# Check remaining target columns and select the uncertainty columns, if any.

for col in dataset_imported.columns:
  if col[:4] == "tar_" and col not in dataset_col_list:
    print(f'uncertainty = "{col}"')
print(f'uncertainty = None')

In [ ]:
uncertainty = "tar_agbd_se"

if uncertainty: dataset_col_list = dataset_col_list + [uncertainty]

## Add covariates

In [ ]:
# Covariates are used as a features in training, but should be handled
# with care for predictions, either selecting a single representative value,
# several values for different scenarios, or averaging across all values for
# a final prediction.

# Check remaining columns and select covariates column
print("covariates  = [")
for col in dataset_imported.columns:
  if col[:4] == "tar_" and col not in dataset_col_list and "geometry" not in col:
    print(f'"{col}",')
print("]")

In [ ]:
covariates  = [
"tar_beam",
"tar_sensitivity",
]

dataset_imported_covar = dataset_imported.copy()

# Rename tar_ to fea_
covariates_renamed = []
if len(covariates)>0:
  covariates_renamed = [covariate.replace('tar','fea') for covariate in covariates]
  dataset_imported_covar.rename(
      columns={i:j for i,j in zip(covariates, covariates_renamed)}, inplace=True
  )

dataset_col_list = dataset_col_list + covariates_renamed

## Select features

In [ ]:
print("selected_features = [")
for col in sorted(dataset_imported_covar.columns):
  if col[:4] == "fea_" and col not in dataset_col_list:
    print(f'  "{col}",')
print("]")

In [ ]:
#AGBD

selected_features = [
  "fea_coast_proximity_km",
  "fea_disturbance_edge_distance_1995",
  "fea_disturbance_edge_distance_1996",
  "fea_disturbance_edge_distance_1997",
  "fea_disturbance_edge_distance_1998",
  "fea_disturbance_edge_distance_1999",
  "fea_disturbance_edge_distance_2000",
  "fea_disturbance_edge_distance_2001",
  "fea_disturbance_edge_distance_2002",
  "fea_disturbance_edge_distance_2003",
  "fea_disturbance_edge_distance_2004",
  "fea_disturbance_edge_distance_2005",
  "fea_disturbance_edge_distance_2006",
  "fea_disturbance_edge_distance_2007",
  "fea_disturbance_edge_distance_2008",
  "fea_disturbance_edge_distance_2009",
  "fea_disturbance_edge_distance_2010",
  "fea_disturbance_edge_distance_2011",
  "fea_disturbance_edge_distance_2012",
  "fea_disturbance_edge_distance_2013",
  "fea_disturbance_edge_distance_2014",
  "fea_disturbance_edge_distance_2015",
  "fea_disturbance_edge_distance_2016",
  "fea_disturbance_edge_distance_2017",
  "fea_disturbance_edge_distance_2018",
  "fea_disturbance_edge_distance_2019",
  "fea_disturbance_edge_distance_2020",
  "fea_disturbance_edge_distance_2021",
  "fea_disturbance_edge_distance_2022",
  "fea_disturbance_edge_distance_2023",
  "fea_disturbance_local_density_1995",
  "fea_disturbance_local_density_1996",
  "fea_disturbance_local_density_1997",
  "fea_disturbance_local_density_1998",
  "fea_disturbance_local_density_1999",
  "fea_disturbance_local_density_2000",
  "fea_disturbance_local_density_2001",
  "fea_disturbance_local_density_2002",
  "fea_disturbance_local_density_2003",
  "fea_disturbance_local_density_2004",
  "fea_disturbance_local_density_2005",
  "fea_disturbance_local_density_2006",
  "fea_disturbance_local_density_2007",
  "fea_disturbance_local_density_2008",
  "fea_disturbance_local_density_2009",
  "fea_disturbance_local_density_2010",
  "fea_disturbance_local_density_2011",
  "fea_disturbance_local_density_2012",
  "fea_disturbance_local_density_2013",
  "fea_disturbance_local_density_2014",
  "fea_disturbance_local_density_2015",
  "fea_disturbance_local_density_2016",
  "fea_disturbance_local_density_2017",
  "fea_disturbance_local_density_2018",
  "fea_disturbance_local_density_2019",
  "fea_disturbance_local_density_2020",
  "fea_disturbance_local_density_2021",
  "fea_disturbance_local_density_2022",
  "fea_disturbance_local_density_2023",
  "fea_forest_edge_distance_1995",
  # "fea_forest_edge_distance_1996",
  # "fea_forest_edge_distance_1997",
  # "fea_forest_edge_distance_1998",
  # "fea_forest_edge_distance_1999",
  # "fea_forest_edge_distance_2000",
  # "fea_forest_edge_distance_2001",
  # "fea_forest_edge_distance_2002",
  # "fea_forest_edge_distance_2003",
  # "fea_forest_edge_distance_2004",
  # "fea_forest_edge_distance_2005",
  # "fea_forest_edge_distance_2006",
  # "fea_forest_edge_distance_2007",
  "fea_forest_edge_distance_2008",
  # "fea_forest_edge_distance_2009",
  # "fea_forest_edge_distance_2010",
  # "fea_forest_edge_distance_2011",
  # "fea_forest_edge_distance_2012",
  # "fea_forest_edge_distance_2013",
  # "fea_forest_edge_distance_2014",
  # "fea_forest_edge_distance_2015",
  # "fea_forest_edge_distance_2016",
  # "fea_forest_edge_distance_2017",
  # "fea_forest_edge_distance_2018",
  # "fea_forest_edge_distance_2019",
  # "fea_forest_edge_distance_2020",
  "fea_forest_edge_distance_2021",
  "fea_forest_edge_distance_2022",
  "fea_forest_edge_distance_2023",
  "fea_forest_local_density_1995",
  # "fea_forest_local_density_1996",
  # "fea_forest_local_density_1997",
  # "fea_forest_local_density_1998",
  # "fea_forest_local_density_1999",
  # "fea_forest_local_density_2000",
  # "fea_forest_local_density_2001",
  # "fea_forest_local_density_2002",
  # "fea_forest_local_density_2003",
  # "fea_forest_local_density_2004",
  # "fea_forest_local_density_2005",
  # "fea_forest_local_density_2006",
  # "fea_forest_local_density_2007",
  "fea_forest_local_density_2008",
  # "fea_forest_local_density_2009",
  # "fea_forest_local_density_2010",
  # "fea_forest_local_density_2011",
  # "fea_forest_local_density_2012",
  # "fea_forest_local_density_2013",
  # "fea_forest_local_density_2014",
  # "fea_forest_local_density_2015",
  # "fea_forest_local_density_2016",
  # "fea_forest_local_density_2017",
  # "fea_forest_local_density_2018",
  # "fea_forest_local_density_2019",
  # "fea_forest_local_density_2020",
  "fea_forest_local_density_2021",
  "fea_forest_local_density_2022",
  "fea_forest_local_density_2023",
  "fea_latitude",
  "fea_longitude",
  "fea_lu_ais_edge_distance",
  "fea_lu_berkelah_jerantut_edge_distance",
  "fea_lu_berkelah_kuantan_edge_distance",
  "fea_lu_berkelah_temerloh_edge_distance",
  "fea_lu_old-growth_protected_areas_edge_distance",
  "fea_lu_remen_chereh_edge_distance",
  "fea_lu_tekai_tembeling_edge_distance",
  "fea_lu_tekam_edge_distance",
  "fea_lu_yong_edge_distance",
  "fea_lu_yong_lipis_edge_distance",
  "fea_topo_dtm_smooth_aspect_cosine",
  "fea_topo_dtm_smooth_aspect_sine",
  "fea_topo_dtm_smooth_circular_variance_aspect_03",
  "fea_topo_dtm_smooth_circular_variance_aspect_07",
  "fea_topo_dtm_smooth_circular_variance_aspect_11",
  "fea_topo_dtm_smooth_deviation_mean_elevation_03",
  "fea_topo_dtm_smooth_deviation_mean_elevation_07",
  "fea_topo_dtm_smooth_deviation_mean_elevation_11",
  "fea_topo_dtm_smooth_eastness",
  "fea_topo_dtm_smooth_elevation",
  "fea_topo_dtm_smooth_northness",
  "fea_topo_dtm_smooth_profile_curvature",
  "fea_topo_dtm_smooth_roughness_03",
  "fea_topo_dtm_smooth_roughness_07",
  "fea_topo_dtm_smooth_roughness_11",
  "fea_topo_dtm_smooth_slope",
  "fea_topo_dtm_smooth_stream_power_index_log10",
  "fea_topo_dtm_smooth_surface_area_ratio",
  "fea_topo_dtm_smooth_tangential_curvature",
  "fea_topo_dtm_smooth_topographic_position_index_03",
  "fea_topo_dtm_smooth_topographic_position_index_07",
  "fea_topo_dtm_smooth_topographic_position_index_11",
  "fea_topo_dtm_smooth_topographic_ruggedness_index",
  "fea_topo_dtm_smooth_topographic_wetness_index",
  "fea_topo_dtm_unsmooth_aspect_cosine",
  "fea_topo_dtm_unsmooth_aspect_sine",
  "fea_topo_dtm_unsmooth_circular_variance_aspect_03",
  "fea_topo_dtm_unsmooth_circular_variance_aspect_07",
  "fea_topo_dtm_unsmooth_circular_variance_aspect_11",
  "fea_topo_dtm_unsmooth_deviation_mean_elevation_03",
  "fea_topo_dtm_unsmooth_deviation_mean_elevation_07",
  "fea_topo_dtm_unsmooth_deviation_mean_elevation_11",
  "fea_topo_dtm_unsmooth_eastness",
  "fea_topo_dtm_unsmooth_elevation",
  "fea_topo_dtm_unsmooth_northness",
  "fea_topo_dtm_unsmooth_profile_curvature",
  "fea_topo_dtm_unsmooth_roughness_03",
  "fea_topo_dtm_unsmooth_roughness_07",
  "fea_topo_dtm_unsmooth_roughness_11",
  "fea_topo_dtm_unsmooth_slope",
  "fea_topo_dtm_unsmooth_stream_power_index_log10",
  "fea_topo_dtm_unsmooth_surface_area_ratio",
  "fea_topo_dtm_unsmooth_tangential_curvature",
  "fea_topo_dtm_unsmooth_topographic_position_index_03",
  "fea_topo_dtm_unsmooth_topographic_position_index_07",
  "fea_topo_dtm_unsmooth_topographic_position_index_11",
  "fea_topo_dtm_unsmooth_topographic_ruggedness_index",
  "fea_topo_dtm_unsmooth_topographic_wetness_index",
]

In [ ]:
# GEDI elevation

# selected_features = [
#   "fea_coast_proximity_km",
#   "fea_disturbance_edge_distance_1990",
#   "fea_disturbance_edge_distance_1991",
#   "fea_disturbance_edge_distance_1992",
#   "fea_disturbance_edge_distance_1993",
#   "fea_disturbance_edge_distance_1994",
#   "fea_disturbance_edge_distance_1995",
#   "fea_disturbance_edge_distance_1996",
#   "fea_disturbance_edge_distance_1997",
#   "fea_disturbance_edge_distance_1998",
#   "fea_disturbance_edge_distance_1999",
#   "fea_disturbance_edge_distance_2000",
#   "fea_disturbance_edge_distance_2001",
#   "fea_disturbance_edge_distance_2002",
#   "fea_disturbance_edge_distance_2003",
#   "fea_disturbance_edge_distance_2004",
#   "fea_disturbance_edge_distance_2005",
#   "fea_disturbance_edge_distance_2006",
#   "fea_disturbance_edge_distance_2007",
#   "fea_disturbance_edge_distance_2008",
#   "fea_disturbance_edge_distance_2009",
#   "fea_disturbance_edge_distance_2010",
#   "fea_disturbance_edge_distance_2011",
#   "fea_disturbance_edge_distance_2012",
#   "fea_disturbance_edge_distance_2013",
#   "fea_disturbance_edge_distance_2014",
#   "fea_disturbance_edge_distance_2015",
#   "fea_disturbance_local_density_1990",
#   "fea_disturbance_local_density_1991",
#   "fea_disturbance_local_density_1992",
#   "fea_disturbance_local_density_1993",
#   "fea_disturbance_local_density_1994",
#   "fea_disturbance_local_density_1995",
#   "fea_disturbance_local_density_1996",
#   "fea_disturbance_local_density_1997",
#   "fea_disturbance_local_density_1998",
#   "fea_disturbance_local_density_1999",
#   "fea_disturbance_local_density_2000",
#   "fea_disturbance_local_density_2001",
#   "fea_disturbance_local_density_2002",
#   "fea_disturbance_local_density_2003",
#   "fea_disturbance_local_density_2004",
#   "fea_disturbance_local_density_2005",
#   "fea_disturbance_local_density_2006",
#   "fea_disturbance_local_density_2007",
#   "fea_disturbance_local_density_2008",
#   "fea_disturbance_local_density_2009",
#   "fea_disturbance_local_density_2010",
#   "fea_disturbance_local_density_2011",
#   "fea_disturbance_local_density_2012",
#   "fea_disturbance_local_density_2013",
#   "fea_disturbance_local_density_2014",
#   "fea_disturbance_local_density_2015",
#   "fea_forest_edge_distance_1990",
#   "fea_forest_edge_distance_2000",
#   "fea_forest_edge_distance_2010",
#   "fea_forest_edge_distance_2011",
#   "fea_forest_edge_distance_2012",
#   "fea_forest_edge_distance_2013",
#   "fea_forest_edge_distance_2014",
#   "fea_forest_edge_distance_2015",
#   "fea_forest_local_density_1990",
#   "fea_forest_local_density_2000",
#   "fea_forest_local_density_2010",
#   "fea_forest_local_density_2011",
#   "fea_forest_local_density_2012",
#   "fea_forest_local_density_2013",
#   "fea_forest_local_density_2014",
#   "fea_forest_local_density_2015",
#   "fea_latitude",
#   "fea_longitude",
#   "fea_lu_ais_edge_distance",
#   "fea_lu_berkelah_jerantut_edge_distance",
#   "fea_lu_berkelah_kuantan_edge_distance",
#   "fea_lu_berkelah_temerloh_edge_distance",
#   "fea_lu_old-growth_protected_areas_edge_distance",
#   "fea_lu_remen_chereh_edge_distance",
#   "fea_lu_tekai_tembeling_edge_distance",
#   "fea_lu_tekam_edge_distance",
#   "fea_lu_yong_edge_distance",
#   "fea_lu_yong_lipis_edge_distance",
#   "fea_topo_dsm_smooth_aspect_cosine",
#   "fea_topo_dsm_smooth_aspect_sine",
#   "fea_topo_dsm_smooth_circular_variance_aspect_03",
#   "fea_topo_dsm_smooth_circular_variance_aspect_07",
#   "fea_topo_dsm_smooth_circular_variance_aspect_11",
#   "fea_topo_dsm_smooth_deviation_mean_elevation_03",
#   "fea_topo_dsm_smooth_deviation_mean_elevation_07",
#   "fea_topo_dsm_smooth_deviation_mean_elevation_11",
#   "fea_topo_dsm_smooth_eastness",
#   "fea_topo_dsm_smooth_elevation",
#   "fea_topo_dsm_smooth_northness",
#   "fea_topo_dsm_smooth_profile_curvature",
#   "fea_topo_dsm_smooth_roughness_03",
#   "fea_topo_dsm_smooth_roughness_07",
#   "fea_topo_dsm_smooth_roughness_11",
#   "fea_topo_dsm_smooth_slope",
#   "fea_topo_dsm_smooth_stream_power_index_log10",
#   "fea_topo_dsm_smooth_surface_area_ratio",
#   "fea_topo_dsm_smooth_tangential_curvature",
#   "fea_topo_dsm_smooth_topographic_position_index_03",
#   "fea_topo_dsm_smooth_topographic_position_index_07",
#   "fea_topo_dsm_smooth_topographic_position_index_11",
#   "fea_topo_dsm_smooth_topographic_ruggedness_index",
#   "fea_topo_dsm_smooth_topographic_wetness_index",
#   "fea_topo_dsm_unsmooth_aspect_cosine",
#   "fea_topo_dsm_unsmooth_aspect_sine",
#   "fea_topo_dsm_unsmooth_circular_variance_aspect_03",
#   "fea_topo_dsm_unsmooth_circular_variance_aspect_07",
#   "fea_topo_dsm_unsmooth_circular_variance_aspect_11",
#   "fea_topo_dsm_unsmooth_deviation_mean_elevation_03",
#   "fea_topo_dsm_unsmooth_deviation_mean_elevation_07",
#   "fea_topo_dsm_unsmooth_deviation_mean_elevation_11",
#   "fea_topo_dsm_unsmooth_eastness",
#   "fea_topo_dsm_unsmooth_elevation",
#   "fea_topo_dsm_unsmooth_northness",
#   "fea_topo_dsm_unsmooth_profile_curvature",
#   "fea_topo_dsm_unsmooth_roughness_03",
#   "fea_topo_dsm_unsmooth_roughness_07",
#   "fea_topo_dsm_unsmooth_roughness_11",
#   "fea_topo_dsm_unsmooth_slope",
#   "fea_topo_dsm_unsmooth_stream_power_index_log10",
#   "fea_topo_dsm_unsmooth_surface_area_ratio",
#   "fea_topo_dsm_unsmooth_tangential_curvature",
#   "fea_topo_dsm_unsmooth_topographic_position_index_03",
#   "fea_topo_dsm_unsmooth_topographic_position_index_07",
#   "fea_topo_dsm_unsmooth_topographic_position_index_11",
#   "fea_topo_dsm_unsmooth_topographic_ruggedness_index",
#   "fea_topo_dsm_unsmooth_topographic_wetness_index",
# ]

In [ ]:
# Sort alphabetically for predictions
selected_features = sorted(selected_features)
dataset_col_list = dataset_col_list + sorted(selected_features)

## Define categorical features

In [ ]:
# Define categorical data for correct model interpretation
print("categorical_columns = [")
for col in dataset_col_list:
  if col.startswith('fea_'):
    print(f'  "{col}",')
print("]")

In [ ]:
categorical_columns = [
  "fea_beam",
]

dataset_imported_cat = dataset_imported_covar.copy()
categorical_mappings = {}
categorical_descriptive_params = []

if len(categorical_columns) > 0:
    for col in categorical_columns:
        if col in dataset_imported_cat.columns:
            # Create tar_ descriptive parameter with original values
            tar_col = col.replace('fea_', 'tar_')
            dataset_imported_cat[tar_col] = dataset_imported_cat[col].copy()
            categorical_descriptive_params.append(tar_col)

            # Convert all categorical features to ordered integers starting from 1
            unique_values = sorted(dataset_imported_cat[col].unique())
            value_to_int = {val: i+1 for i, val in enumerate(unique_values)}
            dataset_imported_cat[col] = dataset_imported_cat[col].map(value_to_int).astype('category')
            categorical_mappings[col] = value_to_int

## Descriptive parameters

In [ ]:
# Check remaining target columns and select parameters that may be useful for descriptive statistics.
print("descriptive_parameters = [")
for col in dataset_imported_cat.columns:
  if col not in dataset_col_list and col not in categorical_descriptive_params:
    print(f'"{col}",')
print("]")

In [ ]:
descriptive_parameters = [

]

# Add categorical descriptive parameters right after target, before features
if 'categorical_descriptive_params' in locals():
    dataset_col_list = dataset_col_list + categorical_descriptive_params

dataset_col_list = dataset_col_list + descriptive_parameters

## Sample size

In [ ]:
# Print current number of rows (data points)
print(f'Number of rows (data points): {len(dataset_imported_cat)}')

In [ ]:
# Randomly sampling the rows of a dataset to generate a smaller subset can
# for testing different parameters. Not intended for final models.

sample_imported_dataset = False
sample_imported_dataset_by_percent = False # If False then by number
sample_imported_dataset_value = 500000 # Set to a percentage or number, or 'None' if not sampling

# Global
dataset_imported_sampled = dataset_imported_cat.copy()

# Sample dataset for testing or HPO
if sample_imported_dataset:
  if sample_imported_dataset_by_percent:
    dataset_imported_sampled = dataset_imported_sampled.sample(frac=sample_imported_dataset_value/100, random_state=1).reset_index().drop(columns=['index'])
  else:
    dataset_imported_sampled = dataset_imported_sampled.sample(n=sample_imported_dataset_value, random_state=1).reset_index().drop(columns=['index'])

print(f'Number of rows (data points) after sampling: {len(dataset_imported_sampled)}')

## Name and compile

In [ ]:
# Set model dataset name (date and time suffix will be added) and sample percent
dataset_name = "agbd_alpha_earth"
# dataset_name = "gedi_elevation"

# Rename ID and descriptive columns
dataset_col_list = [item for item in dataset_col_list if item is not None]
dataset_col_list.sort(key=lambda x: x.startswith('fea_'))
dataset_final = dataset_imported_sampled.copy()[dataset_col_list]
dataset_final.columns = [col.replace('fea_', '').replace('tar_', '') if col == id_column else col for col in dataset_final.columns]

# Model dataset file and directory
local_timezone = timezone(timedelta(hours=8))
model_dataset_name = f"{dataset_name}_{datetime.now(local_timezone).strftime('%y%m%d_%H%M%S')}"
model_dataset_dir = join(models_dir, model_dataset_name)
model_dataset_path = join(model_dataset_dir, model_dataset_name)
model_dataset_description_dir = join(model_dataset_dir, "model_dataset_description.json")

# Check if identical model dataset exists
dataset_exists = False
for subdir, dirs, files in os.walk(models_dir):
  for file in files:
    if file.endswith('.pkl'):
      dataset_existing = pd.read_pickle(join(subdir,file))
      dataset_equals = pd.DataFrame.equals(dataset_existing, dataset_final)
      if dataset_equals:
        dataset_exists = True
        model_dataset = dataset_existing
        model_dataset_dir = subdir
        print(f'An identical model dataset already exists in: {subdir}')
        break
  if dataset_exists:
    break

# Create a model dataset with selected parameters, if it does not exist
if dataset_exists == False:
  makedirs(model_dataset_dir, exist_ok=True)
  dataset_final.to_pickle(f"{model_dataset_path}.pkl")
  model_dataset = pd.read_pickle(f"{model_dataset_path}.pkl")

  # Convert categorical_mappings to JSON-serializable format
  json_categorical_mappings = {}
  if 'categorical_mappings' in locals():
      for col, mapping in categorical_mappings.items():
          json_categorical_mappings[col] = {str(k): int(v) for k, v in mapping.items()}

  model_dataset_description = {
      "model_dataset_name": model_dataset_name,
      "number_of_columns": len(model_dataset.columns),
      "number_of_rows": len(model_dataset),
      "id_column": id_column,
      "selected_target": selected_target,
      "uncertainty": uncertainty,
      "covariates_renamed": covariates_renamed,
      "categorical_features_mappings": json_categorical_mappings,
      "selected_features": selected_features,
      "categorical_columns": categorical_columns,
      "descriptive_parameters": descriptive_parameters,
      "sample_imported_dataset": sample_imported_dataset,
      "sample_imported_dataset_by_percent": sample_imported_dataset_by_percent,
      "sample_imported_dataset_value": sample_imported_dataset_value,
  }
  with open(model_dataset_description_dir, "w") as f:
    f.write(json.dumps(model_dataset_description, indent=2))
  print(f"Model dataset compiled and exported to {model_dataset_dir}")

# Check dataset
model_dataset

# Select model dataset

In [ ]:
# Select an existing model dataset
for subdir in os.listdir(models_dir):
    print(f'selected_model_dataset = "{subdir}"')

In [ ]:
selected_model_dataset = "agbd_251203_161707"

# Select model dataset
model_dataset_dir = join(models_dir,selected_model_dataset)

# Read description for model dataset attributes
with open(join(model_dataset_dir,"model_dataset_description.json")) as model_dataset_description_json:
  model_dataset_description = json.load(model_dataset_description_json)
model_dataset_name = model_dataset_description["model_dataset_name"]
number_of_columns = model_dataset_description["number_of_columns"]
number_of_rows = model_dataset_description["number_of_rows"]
id_column = model_dataset_description["id_column"]
selected_target = model_dataset_description["selected_target"]
uncertainty = model_dataset_description["uncertainty"]
covariates_renamed = model_dataset_description["covariates_renamed"]
selected_features = model_dataset_description["selected_features"] + model_dataset_description["covariates_renamed"]
categorical_features_mappings = model_dataset_description["categorical_features_mappings"]
categorical_columns = model_dataset_description["categorical_columns"]
descriptive_parameters = model_dataset_description["descriptive_parameters"]
sample_imported_dataset = model_dataset_description["sample_imported_dataset"]
sample_imported_dataset_by_percent = model_dataset_description["sample_imported_dataset_by_percent"]
sample_imported_dataset_value = model_dataset_description["sample_imported_dataset_value"]

# Define model dataset description and .json directories
model_description_dir = join(model_dataset_dir, "model_description.json")
final_model_dir = join(model_dataset_dir, "model.json")

# Model parameters

In [ ]:
# WIP categorise_target
categorise_target = False

# Load model dataset
model_dataset = pd.read_pickle(join(f"{model_dataset_dir}/{selected_model_dataset}.pkl"))
model_dataset = model_dataset.copy()

if categorise_target:
  # Create a copy to protect original
  model_dataset_cat = model_dataset.copy()

  # # Categorise per 50 Mg/ha
  # model_dataset_cat[selected_target] = np.where(model_dataset_cat[selected_target] < 50, 0, model_dataset_cat[selected_target])
  # model_dataset_cat[selected_target] = np.where((model_dataset_cat[selected_target] >= 50) & (model_dataset_cat[selected_target] < 100), 1, model_dataset_cat[selected_target])
  # model_dataset_cat[selected_target] = np.where((model_dataset_cat[selected_target] >= 100) & (model_dataset_cat[selected_target] < 150), 2, model_dataset_cat[selected_target])
  # model_dataset_cat[selected_target] = np.where((model_dataset_cat[selected_target] >= 150) & (model_dataset_cat[selected_target] < 200), 3, model_dataset_cat[selected_target])
  # model_dataset_cat[selected_target] = np.where((model_dataset_cat[selected_target] >= 200) & (model_dataset_cat[selected_target] < 250), 4, model_dataset_cat[selected_target])
  # model_dataset_cat[selected_target] = np.where((model_dataset_cat[selected_target] >= 250) & (model_dataset_cat[selected_target] < 300), 5, model_dataset_cat[selected_target])
  # model_dataset_cat[selected_target] = np.where((model_dataset_cat[selected_target] >= 300) & (model_dataset_cat[selected_target] < 350), 6, model_dataset_cat[selected_target])
  # model_dataset_cat[selected_target] = np.where((model_dataset_cat[selected_target] >= 350) & (model_dataset_cat[selected_target] < 400), 7, model_dataset_cat[selected_target])
  # model_dataset_cat[selected_target] = np.where(model_dataset_cat[selected_target] >= 400, 8, model_dataset_cat[selected_target])

  # # Categorise per 100 Mg/ha
  # model_dataset_cat[selected_target] = np.where(model_dataset_cat[selected_target] < 100, 0, model_dataset_cat[selected_target])
  # model_dataset_cat[selected_target] = np.where((model_dataset_cat[selected_target] >= 100) & (model_dataset_cat[selected_target] < 200), 1, model_dataset_cat[selected_target])
  # model_dataset_cat[selected_target] = np.where((model_dataset_cat[selected_target] >= 200) & (model_dataset_cat[selected_target] < 300), 2, model_dataset_cat[selected_target])
  # model_dataset_cat[selected_target] = np.where((model_dataset_cat[selected_target] >= 300) & (model_dataset_cat[selected_target] < 400), 3, model_dataset_cat[selected_target])
  # model_dataset_cat[selected_target] = np.where(model_dataset_cat[selected_target] >= 400, 4, model_dataset_cat[selected_target])

  # Categorise above and below 150 Mg/ha (High Carbon Stock)
  model_dataset_cat[selected_target] = np.where(model_dataset_cat[selected_target] < 150, 0, model_dataset_cat[selected_target])
  model_dataset_cat[selected_target] = np.where(model_dataset_cat[selected_target] >= 150, 1, model_dataset_cat[selected_target])

  # # Categorise species presence
  # model_dataset_cat[selected_target] = np.where(model_dataset_cat[selected_target] == 0, 0, model_dataset_cat[selected_target])
  # model_dataset_cat[selected_target] = np.where(model_dataset_cat[selected_target] == 1, 1, model_dataset_cat[selected_target])

  # Convert to integer for classification
  model_dataset_cat[selected_target] = model_dataset_cat[selected_target].astype(int)

  # Define categories (must be in order and start from 0)
  unique_classes = model_dataset_cat[selected_target].nunique()
  multiclass = unique_classes > 2

  # Ensure multiclass labels are properly encoded from 0 to n-1
  if multiclass:
      unique_vals = sorted(model_dataset_cat[selected_target].unique())
      if unique_vals != list(range(len(unique_vals))):
          label_mapping = {val: i for i, val in enumerate(unique_vals)}
          model_dataset_cat[selected_target] = model_dataset_cat[selected_target].map(label_mapping)
          print(f"Multiclass labels remapped: {dict(zip(range(len(unique_vals)), unique_vals))}")

  # Check for class imbalance in binary classification
  if not multiclass:
      class_counts = model_dataset_cat[selected_target].value_counts()
      imbalance_ratio = class_counts.max() / class_counts.min()

      if imbalance_ratio > 2:
          print(f"Class imbalance detected (ratio: {imbalance_ratio:.2f})")
          # Calculate scale_pos_weight for potential use
          neg_count = class_counts[0] if 0 in class_counts else class_counts.min()
          pos_count = class_counts[1] if 1 in class_counts else class_counts.max()
          scale_pos_weight = neg_count / pos_count
          print(f"Consider scale_pos_weight: {scale_pos_weight:.2f}")

  # Set parameters
  XGBPredictor = xgb.XGBClassifier
  if multiclass:
      objective = 'multi:softprob'
      eval_metric = 'mlogloss'
      num_class = unique_classes  # Required for multiclass in native API
      metric = 'mlogloss'
  else:
      objective = 'binary:logistic'
      eval_metric = 'logloss'
      num_class = None
      metric = 'logloss'
  optimal_value = 'min'
  model_dataset_cat[selected_target] = model_dataset_cat[selected_target].astype("category")
  model_dataset = model_dataset_cat

else:
  XGBPredictor = xgb.XGBRegressor
  objective = 'reg:squarederror'
  eval_metric = 'rmse'
  metric = 'rmse'
  optimal_value = 'min'

# Define optimal values dictionary
optimal_values = {
    "r2": "max", "me": "min", "rmse": "min", "rrmse": "min",
    "accuracy": "max", "precision": "max", "recall": "max", "f1": "max",
    "roc_auc": "max", "roc_auc_ovr": "max", "tss": "max", "mcc": "max",
    "balanced_accuracy": "max", "average_precision": "max", "cohen_kappa": "max",
    "specificity": "max", "logloss": "min", "mlogloss": "min",
    "brier_score": "min"
}

# Define x and y axes for training
model_dataset_x = model_dataset[selected_features].copy()
model_dataset_y = model_dataset[selected_target].copy()

In [ ]:
# Select whether to use the weighted validation score for optimisation, mitigating overfitting.
use_score_weighted = True

# Create dictionary for optimal value and define the weighted score
def def_score_weighted(mean_validation, mean_training, metric_name=None):
    if metric_name is None:
        metric_name = metric
    opt = optimal_values[metric_name]
    if opt == "max":
        return mean_validation - 0.5*max(0, mean_training - mean_validation)
    return mean_validation + 0.5*max(0, mean_validation - mean_training)

In [ ]:
# Exclude specified columns
columns_to_exclude = [
                      # 'fea_sensitivity',
                      # 'fea_latitude',
                      # 'fea_longitude'
                      ]
columns_to_consider = model_dataset_x.columns.drop(columns_to_exclude)

# Calculate max_bin hyperparameter based on the feature with the most unique values
max_unique_col = model_dataset_x[columns_to_consider].nunique().idxmax()
max_bin = model_dataset_x[columns_to_consider].nunique().max()
print(f"{max_unique_col} had the highest number of unique values, max_bin set to {max_bin}")

# Alternatively, set manually by uncommenting:
# max_bin = 256

In [ ]:
# Default HPs are useful for quick testing. Modify the baseline HPs,
# and / or run hyperparameter optimisation for better results.
use_default_hp = True

# Define baseline hyperparameters
baseline_hyperparameters = {
 'tree_method': 'hist',
 'device': 'cuda',
 'enable_categorical': True,
 'max_bin': max_bin,
 'n_estimators': 100000, # Will be limited by early stopping
 'eta': 0.01,
 'early_stopping_rounds': 8,
 'min_child_weight': 71,
 'gamma': 0.175,
 'alpha': 25230,
 'lambda': 37,
 'colsample_bytree': 0.923,
 'colsample_bylevel': 0.83,
 'colsample_bynode': 0.965
}

default_hyperparameters = {
  "tree_method": "hist",
  "device": 'cuda',
  'enable_categorical': True,
  'max_bin': max_bin,
  "objective": objective,
  "eval_metric": eval_metric,
  "n_estimators": 100_000, # Will be limited by early stopping
  "early_stopping_rounds": 10,
}

if use_default_hp: baseline_hyperparameters = default_hyperparameters
else: # Ensure objective and eval_metric are set even when using custom baseline
    if 'objective' not in baseline_hyperparameters: baseline_hyperparameters['objective'] = objective
    if 'eval_metric' not in baseline_hyperparameters: baseline_hyperparameters['eval_metric'] = eval_metric

# Add num_class for multiclass classification
if categorise_target and multiclass:
    baseline_hyperparameters["num_class"] = num_class
    default_hyperparameters["num_class"] = num_class

# Avoids issues using dataframe from CPU
xgb.set_config(verbosity=0, use_rmm=True)

# KFold cross-validation

In [ ]:
# The number of k-folds affects the proportion of data used for training,
# with the complementary proportion used for validation / testing.
# 10 splits = 90 % training, 5 = 80 %, 4 = 75 %, 3 = 67 %, 2 = 50 %.

# Input the number of k-folds to generate train/valid splits.
n_splits = 10

# Number of folds to be used for HPO validation.
# Remaining folds will be used for final testing.
n_hpo_splits = 2

assert n_hpo_splits < n_splits, "n_hpo_splits must be less than n_splits"

In [ ]:
# Use stratified k-fold for classification to maintain class distribution
if categorise_target: kf_split = list(StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1).split(model_dataset_x, model_dataset_y))
else: kf_split = list(KFold(n_splits=n_splits, shuffle=True, random_state=1).split(model_dataset_x, model_dataset_y))

# Return a dictionary full of metrics for calculating accuracy
def calculate_metrics(validation, prediction, prediction_proba=None):
    if not categorise_target:
        # Regression metrics
        validation = validation.astype("float32")
        prediction = prediction.astype("float32")
        metrics = {
            "r2": r2_score(validation, prediction),
            "me": np.mean(np.array(validation).squeeze() - np.array(prediction).squeeze()),
            "rmse": root_mean_squared_error(validation, prediction),
            "rrmse": (root_mean_squared_error(validation, prediction) / np.mean(validation)) * 100 if np.mean(validation) != 0 else np.nan,
            "optimal_value": optimal_values
        }
    else:
        # Classification metrics
        validation = validation.astype("int32")
        prediction = prediction.astype("int32")
        metrics = {
            "accuracy": accuracy_score(validation, prediction),
            "precision": precision_score(validation, prediction, average='weighted', zero_division=0),
            "recall": recall_score(validation, prediction, average='weighted', zero_division=0),
            "f1": f1_score(validation, prediction, average='weighted', zero_division=0),
            "balanced_accuracy": balanced_accuracy_score(validation, prediction),
            "mcc": matthews_corrcoef(validation, prediction),
            "cohen_kappa": cohen_kappa_score(validation, prediction)
        }
        # AUC and probability-based metrics
        if multiclass:
            metrics["roc_auc"] = roc_auc_score(validation, prediction_proba, multi_class='ovr', average='weighted')
            metrics["average_precision"] = average_precision_score(validation, prediction_proba, average='weighted')
        else:
            metrics["roc_auc"] = roc_auc_score(validation, prediction_proba)
            metrics["average_precision"] = average_precision_score(validation, prediction_proba)
        # Calculate specificity and TSS
        if multiclass:
            # Macro-averaged specificity and TSS for multiclass
            cm = confusion_matrix(validation, prediction)
            specificities = []
            sensitivities = []
            for i in range(len(cm)):
                tn = np.sum(cm) - (np.sum(cm[i, :]) + np.sum(cm[:, i]) - cm[i, i])
                fp = np.sum(cm[:, i]) - cm[i, i]
                fn = np.sum(cm[i, :]) - cm[i, i]
                tp = cm[i, i]
                specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
                sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
                specificities.append(specificity)
                sensitivities.append(sensitivity)
            metrics["specificity"] = np.mean(specificities)
            metrics["tss"] = np.mean([sens + spec - 1 for sens, spec in zip(sensitivities, specificities)])
        else:
            # Binary classification
            cm = confusion_matrix(validation, prediction)
            tn, fp, fn, tp = cm.ravel()
            specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
            sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
            metrics["specificity"] = specificity
            metrics["tss"] = sensitivity + specificity - 1
            metrics["brier_score"] = brier_score_loss(validation, prediction_proba)
        # Add logloss
        if multiclass: metrics["mlogloss"] = log_loss(validation, prediction_proba)
        else: metrics["logloss"] = log_loss(validation, prediction_proba)
        metrics["optimal_value"] = optimal_values
    return metrics

def run_kfold(model_dataset, kf_split, predictor, verbose=False, export_test_model=False, test_model_dir=None):
    metrics = []

    # Junk data is used to determine column names by seeing what metrics get calculated
    if categorise_target and multiclass:
        # Use at least as many samples as classes to ensure all classes are represented
        n_junk_samples = max(6, unique_classes)
        junk_data = pd.Series([i % unique_classes for i in range(n_junk_samples)])
        # Create junk probabilities with correct shape (n_samples, num_classes)
        junk_proba = np.full((n_junk_samples, unique_classes), 1.0/unique_classes)
    else: # Binary classification or regression
        junk_data = pd.Series([0, 1, 0, 1, 0, 1])
        junk_proba = np.array([0.5] * 6) if categorise_target else None
    junk_metrics = calculate_metrics(junk_data, junk_data, junk_proba if categorise_target else None)

    cols = []
    for m in [k for k in junk_metrics.keys() if k != 'optimal_value']:
        cols += [f"score_training ({m})", f"score_validation ({m})", f"score_difference ({m})", f"score_weighted ({m})"]
    cols.append("n_estimators")

    fold_index = 1
    for train_index, valid_index in kf_split:
        x_train = model_dataset.loc[train_index][selected_features]
        x_valid = model_dataset.loc[valid_index][selected_features]
        y_train = model_dataset.loc[train_index][selected_target]
        y_valid = model_dataset.loc[valid_index][selected_target]

        dtrain = xgb.DMatrix(x_train, y_train, enable_categorical=True)
        dvalid = xgb.DMatrix(x_valid, y_valid, enable_categorical=True)

        params = predictor.get_params()
        params['eval_metric'] = eval_metric
        if categorise_target and multiclass: params['num_class'] = num_class

        # Sklearn wrapper params handled separately: n_estimators via num_boost_round,
        # Early_stopping_rounds via callback, enable_categorical via DMatrix
        for key in ['n_estimators', 'enable_categorical', 'missing', 'early_stopping_rounds']:
            params.pop(key, None)

        early_stopping = xgb.callback.EarlyStopping(
            rounds=predictor.early_stopping_rounds,
            metric_name=eval_metric,
            data_name='validation_1'
        )

        evals_result = {}
        model = xgb.train(params,
                          dtrain,
                          num_boost_round=predictor.n_estimators,
                          evals=[(dtrain, 'training'), (dvalid, 'validation_1')],
                          callbacks=[early_stopping],
                          evals_result=evals_result,
                          verbose_eval=False)

        # Get predictions
        training_prediction_proba = model.predict(dtrain)
        validation_prediction_proba = model.predict(dvalid)

        if categorise_target:
            # Convert probabilities to class predictions
            if multiclass:
                training_prediction = np.argmax(training_prediction_proba, axis=1)
                validation_prediction = np.argmax(validation_prediction_proba, axis=1)
            else:
                training_prediction = np.round(training_prediction_proba).astype(int)
                validation_prediction = np.round(validation_prediction_proba).astype(int)

            training_metrics = calculate_metrics(y_train, training_prediction, training_prediction_proba)
            validation_metrics = calculate_metrics(y_valid, validation_prediction, validation_prediction_proba)
        else:
            training_metrics = calculate_metrics(y_train, training_prediction_proba)
            validation_metrics = calculate_metrics(y_valid, validation_prediction_proba)

        # Build metrics row
        metrics_row = []
        for m in [k for k in training_metrics.keys() if k != 'optimal_value']:
            score_training = training_metrics[m]
            score_validation = validation_metrics[m]
            score_difference = score_training - score_validation
            score_weighted = def_score_weighted(score_validation, score_training, m)
            metrics_row += [score_training, score_validation, score_difference, score_weighted]

        metrics_row.append(len(evals_result['validation_1'][eval_metric]))
        metrics.append(metrics_row)

        if export_test_model:
            makedirs(test_model_dir, exist_ok=True)
            model_name = f"model_test_fold_{fold_index}"
            metrics_dir = join(test_model_dir, f"{model_name}_metrics.csv")
            model.save_model(join(test_model_dir, f"{model_name}.json"))
            df_fold_metrics = pd.DataFrame(metrics, columns=cols)
            df_fold_metrics.to_csv(metrics_dir)
            print(f"Model export complete for test fold {fold_index}.")

        if verbose:
            score_training = training_metrics[metric]
            score_validation = validation_metrics[metric]
            score_difference = score_training - score_validation
            score_weighted = def_score_weighted(score_validation, score_training)
            print(f"Validation {metric}: {score_validation}")
            print(f"Training {metric}: {score_training}")
            print(f"{metric} difference: {score_difference}")
            print(f"Weighted {metric}: {score_weighted}")
            print(f"--------------------")

        fold_index += 1

    df_kfold_metrics = pd.DataFrame(metrics, columns=cols)
    return df_kfold_metrics

def generate_statistics(df_kfold_metrics):
    mean_training = df_kfold_metrics[f"score_training ({metric})"].mean()
    std_training = df_kfold_metrics[f"score_training ({metric})"].std()
    mean_validation = df_kfold_metrics[f"score_validation ({metric})"].mean()
    std_validation = df_kfold_metrics[f"score_validation ({metric})"].std()
    mean_difference = df_kfold_metrics[f"score_difference ({metric})"].mean()
    std_difference = df_kfold_metrics[f"score_difference ({metric})"].std()
    score_weighted = def_score_weighted(mean_validation, mean_training)
    mean_weighted = df_kfold_metrics[f"score_weighted ({metric})"].mean()
    std_weighted = df_kfold_metrics[f"score_weighted ({metric})"].std()

    print("----------Results Summary----------")
    print(f"Training {metric} mean: {mean_training}")
    print(f"Training {metric} std: {std_training}")
    print(f"Validation {metric} mean: {mean_validation}")
    print(f"Validation {metric} std: {std_validation}")
    print(f"{metric} difference mean: {mean_difference}")
    print(f"{metric} difference std: {std_difference}")
    print(f"Weighted {metric} score: {score_weighted}")

# Default model accuracy

In [ ]:
# Baseline for HPO folds
default_hpo_predictor = XGBPredictor(**default_hyperparameters)
default_hpo = run_kfold(model_dataset, kf_split[:n_hpo_splits], default_hpo_predictor, verbose=True)
generate_statistics(default_hpo)

# Export default_hpo_descr.json
default_hpo_descr_dir = join(model_dataset_dir, "default_hpo_descr.json")
default_hpo_descr = {
  "selected_target": selected_target,
  "hyperparameters": str(default_hyperparameters),
  "metric_used_for_training": metric
}

for col in default_hpo.columns:
  default_hpo_descr[f"avg_{col}"] = float(default_hpo[col].mean())
  default_hpo_descr[f"std_{col}"] = float(default_hpo[col].std())
with open(default_hpo_descr_dir, "w") as file:
  file.write(json.dumps(default_hpo_descr, indent=2))
print("default_hpo_descr.json generation and export complete\n")

In [ ]:
# Default for final testing folds
default_test_predictor = XGBPredictor(**default_hyperparameters)
default_test = run_kfold(model_dataset, kf_split[n_hpo_splits:], default_test_predictor, verbose=True)
generate_statistics(default_test)

# Export default_test_descr.json
default_test_descr_dir = join(model_dataset_dir, "default_test_descr.json")
default_test_descr = {
  "selected_target": selected_target,
  "hyperparameters": str(default_hyperparameters),
  "metric_used_for_training": metric
}

for col in default_test.columns:
  default_test_descr[f"avg_{col}"] = float(default_test[col].mean())
  default_test_descr[f"std_{col}"] = float(default_test[col].std())
with open(default_test_descr_dir, "w") as file:
  file.write(json.dumps(default_test_descr, indent=2))
print("default_test_descr.json generation and export complete\n")

# Hyperparameter optimisation

## Random search

In [ ]:
# Hyperparameters and ranges to randomly sample.
# If uniform or loguniform, subtract the lower range from the upper range.

# Latin Hypercube Sampling provides more uniform coverage of the parameter space
# than completely random sampling. Each HP range is divided into n equal strata
# (where n = iterations), with exactly one sample per stratum. The strata are
# shuffled independently per HP, so combinations remain random while avoiding gaps in coverage.
use_lhs = True
lhs_seed = 1  # Set to None for non-reproducible sampling

# Using a random seed makes the results less variable
use_random_seed = True

hp_distribution = {
    # "max_bin": randint(1000, 2000 + 1),
    "early_stopping_rounds": randint(8, 15 + 1),
    # "eta": uniform(0.01, 0.3 - 0.01),
    "min_child_weight": randint(10, 50 + 1),
    "gamma": uniform(0.0, 0.2 - 0.0),
    "alpha": uniform(10000, 40000 - 10000),
    "lambda": uniform(1, 500 - 1),
    "colsample_bytree": uniform(0.6, 1 - 0.6),
    "colsample_bylevel": uniform(0.9, 1 - 0.9),
    "colsample_bynode": uniform(0.9, 1 - 0.9),
}

rs_hyperparameters = baseline_hyperparameters.copy()
if use_random_seed: rs_hyperparameters["random_state"] = 1

In [ ]:
# Select how many iterations
iterations = 10

# Define directories and paths
hpo_random_search_dir = join(model_dataset_dir, 'hpo_random_search')
hpo_random_search_results_path = join(hpo_random_search_dir, 'iteration_results')
hpo_random_search_best_hyperparameters_dir = join(hpo_random_search_dir, "best_hyperparameters.csv")
Path(hpo_random_search_dir).mkdir(parents=True, exist_ok=True)

# Initialise variables
hpo_random_search_results = pd.DataFrame()
score_col = "score_weighted" if use_score_weighted else "mean_validation"

# Load existing results if available
if exists(f"{hpo_random_search_results_path}.pkl"):
  print("Reading results of the existing trials records...")
  hpo_random_search_results = pd.read_pickle(f"{hpo_random_search_results_path}.pkl")
  print(f"{len(hpo_random_search_results)} records read.")

# Generate LHS samples if enabled
if use_lhs:
  n_completed = len(hpo_random_search_results)
  n_remaining = iterations - n_completed

  if n_remaining > 0:
    if lhs_seed is not None:
      # Deterministic: regenerate identical samples, skip completed ones
      lhs_sampler = LatinHypercube(d=len(hp_distribution), seed=lhs_seed)
      lhs_samples_all = lhs_sampler.random(n=iterations)
      lhs_samples = lhs_samples_all[n_completed:]
    else:
      # Non-deterministic: generate fresh samples for remaining iterations only
      lhs_sampler = LatinHypercube(d=len(hp_distribution), seed=None)
      lhs_samples = lhs_sampler.random(n=n_remaining)

    hp_names = list(hp_distribution.keys())

# Iteratively train models with sampled hyperparameters, saving the values and accuracy scores
current_iteration = len(hpo_random_search_results)
while current_iteration < iterations:
  t_start = datetime.now()

  # Sample HP values using LHS or random sampling
  if use_lhs:
    lhs_idx = current_iteration - (iterations - len(lhs_samples))
    hp_sample = {}
    for i, (hp_name, dist) in enumerate(hp_distribution.items()):
      low, scale = dist.args[0], dist.args[1]
      scaled_value = low + lhs_samples[lhs_idx, i] * scale
      # Check if distribution is integer type
      if hasattr(dist.dist, 'name') and dist.dist.name == 'randint':
        scaled_value = int(np.floor(scaled_value))
      hp_sample[hp_name] = scaled_value
  else:
    hp_sample_random_state = max(1, current_iteration + 1)
    hp_sample = {k: v.rvs(random_state=hp_sample_random_state * (i + 1000000))
                 for i, (k, v) in enumerate(hp_distribution.items())}

  print(f"Trying parameters: {hp_sample}")

  rs_hyperparameters_current = dict(rs_hyperparameters.copy())
  rs_hyperparameters_current.update(hp_sample)
  # Ensure num_class is available for multiclass
  if categorise_target and multiclass: rs_hyperparameters_current["num_class"] = num_class
  rs_predictor = XGBPredictor(**rs_hyperparameters_current)

  df_kfold_metrics = run_kfold(model_dataset, kf_split[:n_hpo_splits], rs_predictor, verbose=False)

  # Generate statistics
  mean_training = df_kfold_metrics[f"score_training ({metric})"].mean()
  mean_validation = df_kfold_metrics[f"score_validation ({metric})"].mean()
  mean_difference = df_kfold_metrics[f"score_difference ({metric})"].mean()
  score_weighted = def_score_weighted(mean_validation, mean_training)
  time_taken = str(datetime.now() - t_start)

  # Compile iteration results
  interim_rs_results = {
      "mean_validation": mean_validation,
      "mean_training": mean_training,
      "mean_difference": mean_difference,
      "score_weighted": score_weighted,
      "time_taken": time_taken,
      "hyperparameters": rs_hyperparameters_current,
  }
  # Add individual HP values for easier analysis
  interim_rs_results.update(hp_sample)

  print(f"Trial completed in {time_taken}.")
  print(f"Mean validation {metric}: {mean_validation:.4f}, Mean training {metric}: {mean_training:.4f}, Mean {metric} difference: {mean_difference:.4f}, Weighted score: {score_weighted:.4f}")

  # Save results
  if len(hpo_random_search_results) == 0:
    hpo_random_search_results = pd.DataFrame([interim_rs_results])
  else:
    hpo_random_search_results = pd.concat([hpo_random_search_results, pd.DataFrame([interim_rs_results])], ignore_index=True)
  hpo_random_search_results.to_pickle(f"{hpo_random_search_results_path}.pkl")
  hpo_random_search_results.to_csv(f"{hpo_random_search_results_path}.csv")

  # Export best hyperparameters
  if optimal_value == "min":
    best_result_id = hpo_random_search_results[score_col].idxmin()
  else:
    best_result_id = hpo_random_search_results[score_col].idxmax()

  best_rs_hyperparameters = hpo_random_search_results.loc[best_result_id]["hyperparameters"].copy()
  export_rs_hyperparameters = best_rs_hyperparameters
  export_rs_hyperparameters[score_col] = hpo_random_search_results.loc[best_result_id][score_col]
  pd.DataFrame(export_rs_hyperparameters, index=[0]).to_csv(hpo_random_search_best_hyperparameters_dir)

  # Update iteration index
  current_iteration += 1

### View RS plots

In [ ]:
# View plots for all Random Search results

# Load results if not already loaded
if 'hpo_random_search_results' not in dir() or hpo_random_search_results.empty:
  hpo_random_search_results = pd.read_pickle(f"{hpo_random_search_results_path}.pkl")

# Extract hyperparameters that were optimised (vary across iterations)
hp_expanded = pd.json_normalize(hpo_random_search_results['hyperparameters'])
optimised_hps = [col for col in hp_expanded.columns if hp_expanded[col].nunique() > 1]

# Filter to only optimised hyperparameters
hpo_x = hp_expanded[optimised_hps]

# Score optimised in Random Search
hpo_y = hpo_random_search_results[['score_weighted']]

# Filter warning "ntree_limit is deprecated, use `iteration_range` or model slicing instead."
warnings.filterwarnings(action='ignore', category=UserWarning)

# Predictor for SHAP interpretation
hpo_predictor = xgb.XGBRegressor()
hpo_predictor.fit(hpo_x, hpo_y)

# SHAP explainer
hpo_explainer = shap.Explainer(hpo_predictor)
hpo_shap_values = hpo_explainer(hpo_x)
shap.plots.beeswarm(hpo_shap_values, plot_size=(20, max(8, len(optimised_hps) * 0.8)))

print(f"Optimised hyperparameters: {optimised_hps}")

In [ ]:
# Individual scatter plots for each optimised hyperparameter
for hp_name in optimised_hps:
  shap.plots.scatter(hpo_shap_values[:, hp_name], color=hpo_shap_values[:, hp_name].data)

## Automated Random Search Evaluation

In [ ]:
# Define directories and paths
hpo_arse_dir = join(model_dataset_dir, "hpo_arse")
hpo_arse_optimisation_plots_dir = join(hpo_arse_dir, "optimisation_plots")
hpo_arse_optimisation_results_dir = join(hpo_arse_dir, "optimisation_results.pkl")
hpo_arse_iteration_results_dir = join(hpo_arse_dir, "iteration_results.pkl")
hpo_arse_best_hyperparameters_dir = join(hpo_arse_dir, "best_hyperparameters.csv")
hpo_arse_cache_shap_dir = join(hpo_arse_dir, "cache_shap.pkl")
hpo_arse_line_graph_dir = join(hpo_arse_dir, "optimisation_results.png")
Path(hpo_arse_dir).mkdir(parents=True, exist_ok=True)
Path(hpo_arse_optimisation_plots_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
# Each optimisation an XGBoost regression model and SHAP values are used to determine
# the hyperparameter with the largest effect on accuracy. This is then narrowed to a
# more optimal distribution based on SHAP effect direction. If the new distribution
# touches the upper or lower bounds of that tested then those bounds are shifted.
# If the mean score of an optimisation is less than the previous, the 'conservative'
# option will retry the former HP range rather than moving to the next round.

# Latin Hypercube Sampling provides more uniform coverage of the parameter space
# than completely random sampling. Each HP range is divided into n equal strata
# (where n = iterations), with exactly one sample per stratum. The strata are
# shuffled independently per HP, so combinations remain random while avoiding gaps in coverage.
use_lhs = True
lhs_seed = 1 # Set to None for non-reproducible sampling

# Using a random seed makes the results less variable, and HPO might not function well
use_random_seed = False

arse_hyperparameters = baseline_hyperparameters.copy()
if use_random_seed: arse_hyperparameters["random_state"] = 1

# Set static ARSE hyperparameters here
# If no range is set (next) then HPO will use these
# arse_hyperparameters["max_bin"] = max_bin
# arse_hyperparameters["early_stopping_rounds"] = 8
# arse_hyperparameters["eta"] = 0.01
# arse_hyperparameters["max_depth"] = 6
# arse_hyperparameters["min_child_weight"] = 71
# arse_hyperparameters["gamma"] = 0.175
# arse_hyperparameters["alpha"] = 25230
# arse_hyperparameters["lambda"] = 37
# arse_hyperparameters["colsample_bytree"] = 0.923
# arse_hyperparameters["colsample_bylevel"] = 0.83
# arse_hyperparameters["colsample_bynode"] = 0.965

# Modify HP ranges to sample here
# Comment out a HP to use the baseline or static HP
hp_distribution = {
    # "max_bin": randint(1500, 2500 + 1),
    "early_stopping_rounds": randint(3, 20 + 1),
    "eta": uniform(0.01, 0.1 - 0.01),
    # "max_depth": randint( 6, 6 + 1),
    "min_child_weight": randint( 1, 50 + 1),
    "gamma": uniform(0.0, 0.1 - 0.0),
    "alpha": uniform(0, 10000 - 0),
    "lambda": uniform(1, 10000 - 1),
    "colsample_bytree": uniform(0.8, 1.0 - 0.8),
    "colsample_bylevel": uniform(0.8, 1.0 - 0.8),
    "colsample_bynode": uniform(0.8, 1.0 - 0.8),
}

df_hp_types = pd.DataFrame(columns=["type", "min", "max"])
df_hp_types.index.name = "hyperparameter"
df_hp_types.loc["max_bin"]                = ["randint", 1, np.inf]
df_hp_types.loc["early_stopping_rounds"]  = ["randint", 1, np.inf]
df_hp_types.loc["eta"]          = ["uniform", 0.01, 1.0]
df_hp_types.loc["max_depth"]  = ["randint", 1, 15]
df_hp_types.loc["min_child_weight"]       = ["randint", 0, np.inf]
df_hp_types.loc["gamma"]                  = ["uniform", 0.0, np.inf]
df_hp_types.loc["alpha"]              = ["uniform", 0.0, np.inf]
df_hp_types.loc["lambda"]             = ["uniform", 1.0, np.inf]
df_hp_types.loc["colsample_bytree"]       = ["uniform", 0.01, 1.0]
df_hp_types.loc["colsample_bylevel"]      = ["uniform", 0.01, 1.0]
df_hp_types.loc["colsample_bynode"]       = ["uniform", 0.01, 1.0]

function_map = {
    "uniform": uniform,
    "randint": randint,
}

In [ ]:
iterations_per_optimisation = 20
total_optimisations = 20
bounds_correction_percent = 50 # Shifts the HP sample range to this % of this sample range if it is at the upper or lower extremes
conservative = False

assert iterations_per_optimisation != len(hp_distribution), 'The number of iterations cannot be the same as the number of hyperparameters sampled (a known issue with shap)'

# Filter warning "ntree_limit is deprecated, use `iteration_range` or model slicing instead."
warnings.filterwarnings(action='ignore', category=UserWarning)

# Initialise variables and directories
shap_data = []
score_col = "score_weighted" if use_score_weighted else "mean_validation"

display_cols = [f"best score ({metric} {score_col})", f"mean score ({metric} {score_col})", "most_important_hp", "new_hp_min", "new_hp_max", f"time_taken (H:M:S)"]
cache_cols = ["hp_distribution_old", "hp_distribution_new"]
hpo_arse_results = pd.DataFrame(columns=display_cols + cache_cols)
hpo_arse_results.index.name = "optimisations"

hpo_arse_iteration_results = pd.DataFrame(columns=["optimisations", "iterations", "hyperparameters", "metric_used", "mean_validation", "mean_training", "mean_difference", "score_weighted", "time_taken", "hp_distribution"])

optimisations = 1
iterations = 1

# Function get the last row where the distribution was updated (if using the conservative flag, this is important for selecting the latest valid row in hpo_arse_results)
roll_back_message = "Rolled back to last valid optimisation."
def get_last_valid_row(df):
  for i in range(len(df) - 1, -1, -1):
    if df.iloc[i]["most_important_hp"] != roll_back_message:
      return df.iloc[i]

# Visualisation function
def display_visualisations(hpo_arse_results, shap_data):
    print(tabulate(hpo_arse_results[display_cols], headers="keys", tablefmt="psql"))

    try:
        # First figure - only for saving the line plot
        fig_line, ax_line = plt.subplots(figsize=(8, 4))
        for score_type in ['best', 'mean']:
            ax_line.plot(hpo_arse_results.index,
                        hpo_arse_results[f"{score_type} score ({metric} {score_col})"],
                        f"{'b' if score_type=='best' else 'r'}o-",
                        label=f'{score_type.capitalize()} score')

        ax_line.set_title(f"{metric} score per optimisation")
        ax_line.set_xlabel("Optimisation")
        ax_line.set_ylabel("Score")
        ax_line.grid(True)
        ax_line.legend()
        ax_line.xaxis.set_major_locator(plt.MaxNLocator(integer=True))
        fig_line.tight_layout()
        fig_line.savefig(hpo_arse_line_graph_dir)
        plt.close(fig_line)  # Close this figure since it's only for saving

        # Now create display figure with all plots
        if shap_data and len(shap_data) > 0:
            i = len(shap_data)
            most_important_hp, shap_values = shap_data[-1]

            # Create figure with 3 subplots for display
            fig_display, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(18, 5))
            fig_display.suptitle(f"Optimisation #{i}", fontsize=16)

            # Line plot for display
            for score_type in ['best', 'mean']:
                ax0.plot(hpo_arse_results.index,
                        hpo_arse_results[f"{score_type} score ({metric} {score_col})"],
                        f"{'b' if score_type=='best' else 'r'}o-",
                        label=f'{score_type.capitalize()} score')
            ax0.set_title(f"{metric} score per optimisation")
            ax0.set_xlabel("Optimisation")
            ax0.set_ylabel("Score")
            ax0.grid(True)
            ax0.legend()
            ax0.xaxis.set_major_locator(plt.MaxNLocator(integer=True))

            # SHAP scatter plot
            shap.plots.scatter(shap_values[:,most_important_hp],
                             color=shap_values[:,most_important_hp].data,
                             ax=ax1, show=False)

            # Feature importance plot
            importances = [np.mean(np.abs(shap_values.values[:, j]))
                         for j in range(shap_values.values.shape[1])]
            feature_importances = dict(sorted(zip(shap_values.feature_names, importances),
                                           key=lambda x: x[1], reverse=True))

            ax2.barh(range(len(feature_importances)), feature_importances.values(), color="#FF0051")
            ax2.set_yticks(range(len(feature_importances)))
            ax2.set_yticklabels(feature_importances.keys())
            ax2.invert_yaxis()
            ax2.set_xlabel("mean(|SHAP value|)")
            ax2.spines["top"].set_visible(False)
            ax2.spines["right"].set_visible(False)

            for j in range(len(feature_importances)):
                ax2.axhline(j+1, color="#888888", lw=0.5, dashes=(1, 5), zorder=-1)
                ax2.text(list(feature_importances.values())[j] + 0.001, j,
                        f"+{list(feature_importances.values())[j]:.2f}",
                        color="#FF0051", ha="left", va="center", fontsize=12)

            fig_display.tight_layout()
            fig_display.savefig(join(hpo_arse_optimisation_plots_dir, f"{i}_shap.png"))
            plt.show()

    except Exception as e:
        print(f"Warning: Error in main visualization: {str(e)}")

# Load cache
if exists(hpo_arse_iteration_results_dir):
  # Load iteration results, set the hyperparameter distribution, and set current process indicies
  hpo_arse_iteration_results = pd.read_pickle(hpo_arse_iteration_results_dir)
  hp_distribution_serialized = hpo_arse_iteration_results.iloc[-1]["hp_distribution"]
  hp_distribution = {k:function_map[df_hp_types.loc[k]["type"]](v[0], v[1]) for k, v in hp_distribution_serialized.items()}

  optimisations = hpo_arse_iteration_results.iloc[-1]["optimisations"]
  iterations = hpo_arse_iteration_results.iloc[-1]["iterations"] + 1
  if iterations > iterations_per_optimisation:
    iterations = 1
    optimisations += 1

  # Load optimisation results, display visualisations, and update the hyperparameter distribution
  if exists(hpo_arse_optimisation_results_dir) and exists(hpo_arse_cache_shap_dir): # May not exit if process stops before the first optimisation finishes
    # Load data
    hpo_arse_results = pd.read_pickle(hpo_arse_optimisation_results_dir)
    with open(hpo_arse_cache_shap_dir, "rb") as data:
      shap_data = pickle.load(data)

    # Update hyperparameter distribution
    hp_distribution_serialized = hpo_arse_results.iloc[-1]["hp_distribution_new"]
    hp_distribution = {k:function_map[df_hp_types.loc[k]["type"]](v[0], v[1]) for k, v in hp_distribution_serialized.items()}

    # Display visualisations
    display_visualisations(hpo_arse_results, shap_data)

# Iteratively train models with sampled hyper parameters, saving the values and metric score
while optimisations <= total_optimisations:
  iterations_progress_label = widgets.Label(value=f"Iteration of current optimisation: {iterations}/{iterations_per_optimisation}")
  display(iterations_progress_label)
  hp_distribution_old = hp_distribution.copy()
  hp_distribution_new = hp_distribution.copy() # Copy old distribution for now, update this value later
  hp_distribution_old_serialized = {k:v.args for k, v in hp_distribution_old.items()}

  # Generate LHS samples for this optimisation round if enabled
  if use_lhs:
    round_seed = lhs_seed * optimisations if lhs_seed else None
    resuming_mid_optimisation = iterations > 1

    if round_seed is not None:
      # Deterministic: regenerate identical samples
      lhs_sampler = LatinHypercube(d=len(hp_distribution), seed=round_seed)
      lhs_samples = lhs_sampler.random(n=iterations_per_optimisation)
    elif resuming_mid_optimisation:
      # Non-deterministic resume: generate fresh LHS for remaining iterations only
      completed_iterations = hpo_arse_iteration_results[
          hpo_arse_iteration_results["optimisations"] == optimisations
      ]
      n_completed = len(completed_iterations)
      n_remaining = iterations_per_optimisation - n_completed

      lhs_sampler = LatinHypercube(d=len(hp_distribution), seed=None)
      lhs_samples_remaining = lhs_sampler.random(n=n_remaining)

      # Create full array with NaN placeholders for completed iterations
      lhs_samples = np.full((iterations_per_optimisation, len(hp_distribution)), np.nan)
      lhs_samples[n_completed:] = lhs_samples_remaining
    else:
      # Fresh start without seed
      lhs_sampler = LatinHypercube(d=len(hp_distribution), seed=None)
      lhs_samples = lhs_sampler.random(n=iterations_per_optimisation)

    hp_names = list(hp_distribution.keys())

  while iterations <= iterations_per_optimisation:
    t_start = datetime.now()

    # Sample HP values using LHS or random sampling
    if use_lhs:
      hp_sample = {}
      for i, (hp_name, dist) in enumerate(hp_distribution.items()):
        low, scale = dist.args[0], dist.args[1]
        scaled_value = low + lhs_samples[iterations - 1, i] * scale
        if df_hp_types.loc[hp_name]['type'] == 'randint':
          scaled_value = int(np.floor(scaled_value))
        hp_sample[hp_name] = scaled_value
    else:
      hp_sample_random_state = len(hpo_arse_iteration_results) + 1
      hp_sample = {k: v.rvs(random_state=hp_sample_random_state * (i + 1) + optimisations * 1000)
                   for i, (k, v) in enumerate(hp_distribution.items())}

    sampled_parameters = hp_sample
    arse_hyperparameters = dict(arse_hyperparameters.copy())
    arse_hyperparameters.update(sampled_parameters)
    # Ensure num_class is available for multiclass
    if categorise_target and multiclass: arse_hyperparameters["num_class"] = num_class
    iterations_progress_label.value = f"Iteration of current optimisation: {iterations}/{iterations_per_optimisation}"

    arse_predictor = XGBPredictor(**arse_hyperparameters)
    df_kfold_metrics = run_kfold(model_dataset, kf_split[:n_hpo_splits], arse_predictor)

    # Generate statistics
    mean_training = df_kfold_metrics[f"score_training ({metric})"].mean()
    mean_validation = df_kfold_metrics[f"score_validation ({metric})"].mean()
    mean_difference = df_kfold_metrics[f"score_difference ({metric})"].mean()
    score_weighted = def_score_weighted(mean_validation, mean_training)

    # Save results
    time_taken = str(datetime.now() - t_start)
    i = len(hpo_arse_iteration_results)
    hpo_arse_iteration_results.loc[i + 1] = [optimisations, iterations, arse_hyperparameters, metric, mean_validation, mean_training, mean_difference, score_weighted, time_taken, hp_distribution_old_serialized]
    hpo_arse_iteration_results.to_pickle(hpo_arse_iteration_results_dir)
    hpo_arse_iteration_results.to_csv(f"{hpo_arse_iteration_results_dir[:-4]}.csv")

    # Export best hyperparameters
    if optimal_value == "min":
      best_result_id = hpo_arse_iteration_results[score_col].idxmin()
    else:
      best_result_id = hpo_arse_iteration_results[score_col].idxmax()

    best_arse_hyperparameters = hpo_arse_iteration_results.loc[best_result_id]["hyperparameters"].copy()

    export_arse_hyperparameters = best_arse_hyperparameters
    export_arse_hyperparameters[score_col] = hpo_arse_iteration_results.loc[best_result_id][score_col]
    pd.DataFrame(export_arse_hyperparameters, index=[0]).to_csv(hpo_arse_best_hyperparameters_dir)

    # Update iteration index
    iterations += 1

  iterations = 1

  # Collect results of only this optimisation round, expanding hyperparameters
  interim_results = hpo_arse_iteration_results.loc[hpo_arse_iteration_results["optimisations"] == optimisations].copy()
  hp_expanded = pd.DataFrame(interim_results["hyperparameters"].tolist(), index=interim_results.index)
  interim_results = pd.concat([interim_results[["time_taken", score_col]], hp_expanded[list(hp_distribution.keys())]], axis=1)

  # Generate SHAP values
  hpo_x = interim_results[list(hp_distribution.keys())]
  hpo_x = hpo_x.apply(pd.to_numeric)
  hpo_y = interim_results[score_col]
  hpo_y = hpo_y.apply(pd.to_numeric)
  hp_predictor = xgb.XGBRegressor()
  hp_predictor.fit(hpo_x, hpo_y)
  explainer = shap.TreeExplainer(hp_predictor)
  shap_values = explainer(hpo_x)

  # Calculate the feature importance (mean absolute shap value) for each feature, then select the most important
  importances = []
  for i in range(shap_values.values.shape[1]):
    importances.append(np.mean(np.abs(shap_values.values[:, i])))
  feature_importances = {fea: imp for imp, fea in zip(importances, hpo_x.columns)}
  feature_importances = {k: v for k, v in sorted(feature_importances.items(), key=lambda item: item[1], reverse = True)}

  most_important_hp = max(feature_importances, key=feature_importances.get)
  try:
    most_important_hp_shap = shap_values[:,most_important_hp]
  except:
    most_important_hp_shap = shap_values[most_important_hp,:]

  # Break loop if most important feature has no effect (optimisation complete)
  if len((most_important_hp_shap.values!=0).nonzero()[0]) == 0:
    print("Optimisation complete")
    break

  # Calculate quartile boundaries for the most important HP
  hp_p25, hp_p50, hp_p75 = np.percentile(hpo_x[most_important_hp], [25, 50, 75])

  # Create inclusive masks for each quartile
  quartile_masks = [
      most_important_hp_shap.data <= hp_p25,
      (most_important_hp_shap.data >= hp_p25) & (most_important_hp_shap.data <= hp_p50),
      (most_important_hp_shap.data >= hp_p50) & (most_important_hp_shap.data <= hp_p75),
      most_important_hp_shap.data >= hp_p75
  ]

  # Calculate mean SHAP values for each quartile with zero division protection
  shap_quartile_means = np.array([
      np.mean(most_important_hp_shap.values[mask]) if mask.sum() > 0 else np.nan
      for mask in quartile_masks
  ])

  # Select optimal quartile based on min or max optimal value
  if optimal_value == "min":
    best_quartile_idx = np.nanargmin(shap_quartile_means)
  else:
    best_quartile_idx = np.nanargmax(shap_quartile_means)

  optimal_hp_values = most_important_hp_shap.data[quartile_masks[best_quartile_idx]]
  lower_limit_optimal = (best_quartile_idx == 0)
  upper_limit_optimal = (best_quartile_idx == 3)

  # Get HP type for bounds handling
  hp_type = df_hp_types.loc[most_important_hp]['type']
  hp_is_int = (hp_type == "randint")

  # Get new min and max optimal values
  new_hp_min = np.min(optimal_hp_values)
  new_hp_max = np.max(optimal_hp_values)
  if hp_is_int:
    new_hp_min = np.trunc(new_hp_min)
    new_hp_max = np.ceil(new_hp_max)

  # Calculate range correction as percentage of tested range
  range_correction = bounds_correction_percent * np.ptp(most_important_hp_shap.data) / 100

  # Extend bounds if optimal values are at the edge of tested range
  if upper_limit_optimal:
    new_hp_max = min(new_hp_max + range_correction, df_hp_types.loc[most_important_hp]['max'])
  if lower_limit_optimal:
    new_hp_min = max(new_hp_min - range_correction, df_hp_types.loc[most_important_hp]['min'])

  # Round after bounds correction for integer HPs
  if hp_is_int:
    new_hp_min = np.trunc(new_hp_min)
    new_hp_max = np.ceil(new_hp_max)

  # Update and export SHAP data
  shap_data.append([most_important_hp, shap_values])
  with open(hpo_arse_cache_shap_dir, "wb") as f:
    pickle.dump(shap_data, f)

  # Update and export optimisation results
  if optimal_value == "min":
    best_result_id = interim_results[score_col].idxmin()
  else:
    best_result_id = interim_results[score_col].idxmax()

  i = len(hpo_arse_results)
  time_taken = 0
  for index, value in interim_results["time_taken"].items():
    pt = datetime.strptime(value, "%H:%M:%S.%f")
    time_taken += pt.second + pt.minute*60 + pt.hour*3600
  time_taken = str(timedelta(seconds=time_taken))

  roll_back_hp = False
  if conservative and optimisations > 1:
    avg_score_col = [col for col in hpo_arse_results.columns if col.startswith("mean score")]
    current_mean = interim_results[score_col].mean()
    previous_mean = get_last_valid_row(hpo_arse_results)[avg_score_col][0] # Compare mean to the last valid optimisation, not necessarily the previous optimisation
    if (optimal_value == "min" and current_mean > previous_mean) or (optimal_value == "max" and current_mean < previous_mean):
      roll_back_hp = True

  if roll_back_hp:
    hp_distribution_new_serialized = get_last_valid_row(hpo_arse_results)["hp_distribution_old"]
    hp_distribution_new = {k:function_map[df_hp_types.loc[k]["type"]](v[0], v[1]) for k, v in hp_distribution_new_serialized.items()}
    most_important_hp = roll_back_message
    new_hp_min = np.nan
    new_hp_max = np.nan
  else:
    if hp_is_int:
      hp_distribution_new[most_important_hp] = randint(int(new_hp_min), int(new_hp_max) + 1)
    else:
      hp_distribution_new[most_important_hp] = uniform(new_hp_min, new_hp_max - new_hp_min)

  hp_distribution = hp_distribution_new.copy()
  hp_distribution_new_serialized = {k:v.args for k, v in hp_distribution_new.items()}

  hpo_arse_results.loc[i + 1] = {
      f"best score ({metric} {score_col})": interim_results.loc[best_result_id][score_col],
      f"mean score ({metric} {score_col})": interim_results[score_col].mean(),
      "most_important_hp": most_important_hp,
      "new_hp_min": new_hp_min,
      "new_hp_max": new_hp_max,
      f"time_taken (H:M:S)": time_taken,
      "hp_distribution_old": hp_distribution_old_serialized,
      "hp_distribution_new": hp_distribution_new_serialized,
  }
  hpo_arse_results.to_pickle(hpo_arse_optimisation_results_dir)
  hpo_arse_results.to_csv(f"{hpo_arse_optimisation_results_dir[:-4]}.csv") # Human readable

  # Update visualisations
  display_visualisations(hpo_arse_results, shap_data)

  # Update optimisations index
  optimisations += 1

### View ARSE plots

In [ ]:
# View plots for all ARSE results

# Load results if not already loaded
if 'hpo_arse_iteration_results' not in dir() or hpo_arse_iteration_results.empty:
  hpo_arse_dir = join(model_dataset_dir, "hpo_arse")
  hpo_arse_iteration_results_dir = join(hpo_arse_dir, "iteration_results.pkl")
  hpo_arse_iteration_results = pd.read_pickle(hpo_arse_iteration_results_dir)

# Extract hyperparameters that were optimised (vary across iterations)
hp_expanded = pd.json_normalize(hpo_arse_iteration_results['hyperparameters'])
optimised_hps = [col for col in hp_expanded.columns if hp_expanded[col].nunique() > 1]

# Filter to only optimised hyperparameters
arse_hpo_x = hp_expanded[optimised_hps]

# Score optimised in ARSE
arse_hpo_y = hpo_arse_iteration_results[['score_weighted']]

# Predictor for SHAP interpretation
arse_hpo_predictor = xgb.XGBRegressor()
arse_hpo_predictor.fit(arse_hpo_x, arse_hpo_y)

# SHAP explainer
arse_hpo_explainer = shap.Explainer(arse_hpo_predictor)
arse_hpo_shap_values = arse_hpo_explainer(arse_hpo_x)
shap.plots.beeswarm(arse_hpo_shap_values, plot_size=(20, max(8, len(optimised_hps) * 0.8)))

print(f"Optimised hyperparameters: {optimised_hps}")

In [ ]:
# Individual scatter plots for each optimised hyperparameter
for hp_name in optimised_hps:
  shap.plots.scatter(arse_hpo_shap_values[:, hp_name], color=arse_hpo_shap_values[:, hp_name].data)

# Final model

## Define and test

In [ ]:
# Using a random seed makes the results replicable
use_random_seed = True

# Set to True to use the 'best hyperparameters' from HPO
# Set to false to use hyperparameters in the code block above
use_best_hyperparameters = False
hpo_method = "hpo_arse" # Options: "hpo_arse", "hpo_random_search"

# Use default hyperparameters
test_hyperparameters = default_hyperparameters.copy()

# Manually define final hyperparameters
# GEDI elevation
# test_hyperparameters = {
#  'tree_method': 'hist',
#  'device': 'cuda',
#  'enable_categorical': True,
#  'max_bin': max_bin,
#  'n_estimators': 100000, # Will be limited by early stopping
#  'eta': 0.01,
#  'early_stopping_rounds': 18,
#  'min_child_weight': 29,
#  'gamma': 0.05,
#  'alpha': 1480,
#  'lambda': 2150,
#  'colsample_bytree': 0.98,
#  'colsample_bylevel': 1,
#  'colsample_bynode': 1
# }

# # AGBD Alpha Earth
# test_hyperparameters = {
#  'tree_method': 'hist',
#  'device': 'cuda',
#  'enable_categorical': True,
#  'max_bin': 256,
#  'n_estimators': 100000, # Will be limited by early stopping
#  'eta': 0.05,
#  'early_stopping_rounds': 3,
#  'min_child_weight': 55,
#  'gamma': 0.018,
#  'alpha': 83,
#  'lambda': 10500,
#  'colsample_bytree': 0.57,
#  'colsample_bylevel': 0.53,
#  'colsample_bynode': 0.5
# }

if use_random_seed: test_hyperparameters["random_state"] = 1

In [ ]:
# Test model
test_model_dir = join(model_dataset_dir,"model_test")

# If optimised hyperparameters exist, use them
if use_best_hyperparameters:
  test_hyperparameters = pd.read_csv(join(model_dataset_dir, hpo_method, "best_hyperparameters.csv"), index_col=0).to_dict(orient="records")[0]
  test_hyperparameters = {k: v for k, v in test_hyperparameters.items() if "score_" not in k}

predictor = XGBPredictor(**test_hyperparameters)

assert not exists(model_description_dir) and not exists(final_model_dir), "Remove both \"model_description.json\" and \"model.json\", before exporting a new model"

# Run model
df_kfold_metrics = run_kfold(model_dataset, kf_split[n_hpo_splits:], predictor, verbose=True,
                             export_test_model=True, test_model_dir=test_model_dir)

generate_statistics(df_kfold_metrics)

model_description = {
    "metric_used_for_training": metric,
    "optimal_value": optimal_value,
    "use_score_weighted": use_score_weighted,
    "n_splits": n_splits, # Number of k-fold splits
    "splits_hpo": n_hpo_splits, # Number of k-fold splits set aside for HPO, the remaining are used for final testing.
    "hyperparameters": str(test_hyperparameters)
}

for col in df_kfold_metrics.columns:
  model_description[f"{col} mean"] = float(df_kfold_metrics[col].mean())
  model_description[f"{col} std"] = float(df_kfold_metrics[col].std())

# Export model_description.json
with open(model_description_dir, "w") as f:
  f.write(json.dumps(model_description, indent=2))
print("model_description.json generation and export complete.")

## Descriptive plots

In [ ]:
# Descriptive plots

# Assert model description exists
assert exists(model_description_dir), "\"model_description.json\" must exist to continue, run previous cells"

# Reload hyperparameters
with open(model_description_dir) as model_description_json:
  model_description = json.load(model_description_json)
final_hyperparameters = ast.literal_eval(model_description["hyperparameters"])

# Remove early stopping and replace with mean n_estimators
if "early_stopping_rounds" in final_hyperparameters:
  final_hyperparameters = {k:v for k, v in final_hyperparameters.items() if k != "early_stopping_rounds"}
  final_hyperparameters["n_estimators"] = round(model_description["n_estimators mean"])

# Select the last k-split
train_index_plots, valid_index_plots = kf_split[n_splits-1]
x_train_plots = model_dataset.loc[train_index_plots][selected_features]
x_valid_plots = model_dataset.loc[valid_index_plots][selected_features]
y_train_plots = model_dataset.loc[train_index_plots][selected_target]
y_valid_plots = model_dataset.loc[valid_index_plots][selected_target]

# Create DMatrix objects
dtrain_plots = xgb.DMatrix(x_train_plots, y_train_plots, enable_categorical=True)
dvalid_plots = xgb.DMatrix(x_valid_plots, y_valid_plots, enable_categorical=True)

# Set up parameters
predictor = XGBPredictor(**final_hyperparameters)
params_plots = predictor.get_params()
params_plots['eval_metric'] = eval_metric
# Default fix for new XGBoost version
for key in ['n_estimators', 'enable_categorical', 'missing']: params_plots.pop(key, None)

# Train model
evals_result = {}
model_plots = xgb.train(params_plots,
                        dtrain_plots,
                        num_boost_round=final_hyperparameters['n_estimators'],
                        evals=[(dtrain_plots, 'training'), (dvalid_plots, 'validation_1')],
                        evals_result=evals_result,
                        verbose_eval=False)

# Feature importances
feature_importances = model_plots.get_score(importance_type='weight')
sorted_importances = sorted(feature_importances.items(), key=lambda x: x[1], reverse=True)
features, importances = zip(*sorted_importances)

plt.figure(figsize=(10, 30))
plt.barh(features, importances, align='center')
plt.xlabel('Importance')
plt.title('Feature Importances')
plt.gca().invert_yaxis()  # Invert y-axis to have the most important feature at the top
plt.show()

# Prediction versus test data plot
validation_prediction = model_plots.predict(dvalid_plots)
# Convert predictions to class labels for classification
if categorise_target:
    # For multiclass, use argmax to get predicted class from probabilities
    if multiclass: validation_prediction = np.argmax(validation_prediction, axis=1)
    # For binary, round probabilities to get class labels
    else: validation_prediction = np.round(validation_prediction).astype(int)
validation_prediction_series = pd.Series(validation_prediction, index=y_valid_plots.index, name=y_valid_plots.name)

plt.figure(figsize=(12, 7))
if categorise_target:
    # For classification, use discrete bins for categories
    if hasattr(y_valid_plots, 'cat'): actual_values = y_valid_plots.cat.codes
    else: actual_values = y_valid_plots.astype(int)
    # For multiclass, validation_prediction already contains class labels from argmax
    # For binary, it contains probabilities that need rounding
    if multiclass: predicted_values = validation_prediction.astype(int)
    else: predicted_values = np.round(validation_prediction).astype(int)

    # Get unique classes for bins
    all_classes = sorted(set(actual_values) | set(predicted_values))
    bins = np.arange(min(all_classes), max(all_classes) + 2) - 0.5

    plt.hist(actual_values, bins=bins, label=f'{y_valid_plots.name} (actual)', alpha=0.6, align='mid')
    plt.hist(predicted_values, bins=bins, label=f'{y_valid_plots.name} (predicted)', alpha=0.6, align='mid')
    plt.xticks(all_classes)
    plt.xlabel('Class')
else:
    # For regression, use continuous bins
    bins = np.linspace(y_valid_plots.min(), y_valid_plots.max(), 100)
    y_valid_plots.hist(bins=bins, label=f'{y_valid_plots.name} (actual)', alpha=0.6)
    validation_prediction_series.hist(bins=bins, label=f'{y_valid_plots.name} (predicted)', alpha=0.6)
    plt.xlabel('Value')

plt.legend()
plt.ylabel('Frequency')
plt.title('Prediction vs. Actual Data')
plt.show()

## Train and export with full dataset

In [ ]:
# Generate final model

# Assert exists
assert exists(model_description_dir), "\"model_description.json\" must exist to continue, run previous cells"

# Reload hyperparameters
with open(model_description_dir) as model_description_json:
  model_description = json.load(model_description_json)
final_hyperparameters = ast.literal_eval(model_description["hyperparameters"])

# Remove early stopping and replace with mean n_estimators
if "early_stopping_rounds" in final_hyperparameters:
  final_hyperparameters = {k:v for k, v in final_hyperparameters.items() if k != "early_stopping_rounds"}
  final_hyperparameters["n_estimators"] = round(model_description["n_estimators mean"])


# Create DMatrix objects
dtrain_final = xgb.DMatrix(model_dataset_x, model_dataset_y, enable_categorical=True)

# Train model on full dataset
predictor = XGBPredictor(**final_hyperparameters)
params_final = predictor.get_params()
params_final['eval_metric'] = eval_metric
# sklearn wrapper params handled separately via num_boost_round and DMatrix
for key in ['n_estimators', 'enable_categorical', 'missing']: params_final.pop(key, None)
model_final = xgb.train(params_final,
                        dtrain_final,
                        num_boost_round=final_hyperparameters['n_estimators'],
                        verbose_eval=True)

# Export model
model_final.save_model(final_model_dir)
print("Model training and 'model.json' export complete.")

# Model interpretation (SHAP)

In [ ]:
# Define and create directories
shap_dir = join(model_dataset_dir, "shap")
shap_feature_effect_plots_dir = join(shap_dir, 'feature_effect_plots')
shap_cache_dir = join(shap_dir, 'shap_cache')

os.makedirs(shap_dir, exist_ok=True)
os.makedirs(shap_feature_effect_plots_dir, exist_ok=True)
os.makedirs(shap_cache_dir, exist_ok=True)

# Print the number of features
num_features = len(selected_features)
print(f"# Number of features: {num_features}\n")

def modify_feature_name(feature):
    # First capture if it's smoothed/unsmoothed
    is_unsmoothed = 'unsmooth_' in feature
    is_smoothed = 'smooth_' in feature and not is_unsmoothed
    # Remove smoothing indicators (we'll add them back at the end)
    if is_unsmoothed:
      feature = feature.replace('unsmooth_', '')
      feature = feature.replace('un', '', 1)
    if is_smoothed:
      feature = feature.replace('smooth_', '')
      feature = feature.replace('un', '', 1)
    # Remove specified prefixes from the feature name
    prefixes = ['fea_', 'topo_', 'dtm_', 'dsm_', 'lu_', 'pa_']
    for prefix in prefixes: feature = feature.replace(prefix, '')
    # Replace specific patterns
    feature = feature.replace('beam', 'GEDI beam')
    feature = feature.replace('sensitivity', 'GEDI sensitivity')
    feature = feature.replace('disturbance_edge_distance', 'Disturbance with edge distance')
    feature = feature.replace('_km', ' (km)')
    feature = feature.replace('_03', ' (3 pixel)')
    feature = feature.replace('_07', ' (7 pixel)')
    feature = feature.replace('_11', ' (11 pixel)')
    # Specific to Tekai landscape, Malaysia
    feature = feature.replace('ais_edge_distance', 'Ais forest reserves with edge distance')
    feature = feature.replace('berkelah_jerantut_edge_distance', 'Berkelah Jerantut Forest Reserve with edge distance')
    feature = feature.replace('berkelah_kuantan_edge_distance', 'Berkelah Kuantan forest reserves with edge distance')
    feature = feature.replace('berkelah_temerloh_edge_distance', 'Berkelah Temerloh Forest Reserve with edge distance')
    feature = feature.replace('old-growth_protected_areas_edge_distance', 'Old-growth protected areas with edge distance')
    feature = feature.replace('remen_chereh_edge_distance', 'Remen Chereh forest reserves ewith edge distance')
    feature = feature.replace('tekai_tembeling_edge_distance', 'Tekai Tembeling forest reserves with edge distance')
    feature = feature.replace('tekam_edge_distance', 'Tekam Forest Reserve edge distance')
    feature = feature.replace('yong_edge_distance', 'Yong Lipis forest reserves with edge distance')
    feature = feature.replace('yong_lipis_edge_distance', 'Yong forest reserves with edge distance')
    # Replace remaining underscores with spaces
    feature = feature.replace('_', ' ')
    # Add smoothing term at the end
    if is_unsmoothed: feature = feature + ' unsmoothed'
    elif is_smoothed: feature = feature + ' smoothed'
    # Capitalize and clean up
    feature = feature.strip()
    feature = feature[0].upper() + feature[1:]
    return feature

# Generate a dictionary of original feature names mapped to modified names
# Copy and paste the following 'feature_name_mapping' dictionary and modify it as needed
print("feature_name_mapping = {")
for feature in selected_features:
    modified_name = modify_feature_name(feature)
    print(f"    '{feature}': '{modified_name}',")
print("}")

In [ ]:
## GEDI elevation

# # Number of features: 131

# feature_name_mapping = {
#     'fea_coast_proximity_km': 'Coast proximity (km)',
#     'fea_disturbance_edge_distance_1990': 'Disturbance with edge distance 1990',
#     'fea_disturbance_edge_distance_1991': 'Disturbance with edge distance 1991',
#     'fea_disturbance_edge_distance_1992': 'Disturbance with edge distance 1992',
#     'fea_disturbance_edge_distance_1993': 'Disturbance with edge distance 1993',
#     'fea_disturbance_edge_distance_1994': 'Disturbance with edge distance 1994',
#     'fea_disturbance_edge_distance_1995': 'Disturbance with edge distance 1995',
#     'fea_disturbance_edge_distance_1996': 'Disturbance with edge distance 1996',
#     'fea_disturbance_edge_distance_1997': 'Disturbance with edge distance 1997',
#     'fea_disturbance_edge_distance_1998': 'Disturbance with edge distance 1998',
#     'fea_disturbance_edge_distance_1999': 'Disturbance with edge distance 1999',
#     'fea_disturbance_edge_distance_2000': 'Disturbance with edge distance 2000',
#     'fea_disturbance_edge_distance_2001': 'Disturbance with edge distance 2001',
#     'fea_disturbance_edge_distance_2002': 'Disturbance with edge distance 2002',
#     'fea_disturbance_edge_distance_2003': 'Disturbance with edge distance 2003',
#     'fea_disturbance_edge_distance_2004': 'Disturbance with edge distance 2004',
#     'fea_disturbance_edge_distance_2005': 'Disturbance with edge distance 2005',
#     'fea_disturbance_edge_distance_2006': 'Disturbance with edge distance 2006',
#     'fea_disturbance_edge_distance_2007': 'Disturbance with edge distance 2007',
#     'fea_disturbance_edge_distance_2008': 'Disturbance with edge distance 2008',
#     'fea_disturbance_edge_distance_2009': 'Disturbance with edge distance 2009',
#     'fea_disturbance_edge_distance_2010': 'Disturbance with edge distance 2010',
#     'fea_disturbance_edge_distance_2011': 'Disturbance with edge distance 2011',
#     'fea_disturbance_edge_distance_2012': 'Disturbance with edge distance 2012',
#     'fea_disturbance_edge_distance_2013': 'Disturbance with edge distance 2013',
#     'fea_disturbance_edge_distance_2014': 'Disturbance with edge distance 2014',
#     'fea_disturbance_edge_distance_2015': 'Disturbance with edge distance 2015',
#     'fea_disturbance_local_density_1990': 'Disturbance local density 1990',
#     'fea_disturbance_local_density_1991': 'Disturbance local density 1991',
#     'fea_disturbance_local_density_1992': 'Disturbance local density 1992',
#     'fea_disturbance_local_density_1993': 'Disturbance local density 1993',
#     'fea_disturbance_local_density_1994': 'Disturbance local density 1994',
#     'fea_disturbance_local_density_1995': 'Disturbance local density 1995',
#     'fea_disturbance_local_density_1996': 'Disturbance local density 1996',
#     'fea_disturbance_local_density_1997': 'Disturbance local density 1997',
#     'fea_disturbance_local_density_1998': 'Disturbance local density 1998',
#     'fea_disturbance_local_density_1999': 'Disturbance local density 1999',
#     'fea_disturbance_local_density_2000': 'Disturbance local density 2000',
#     'fea_disturbance_local_density_2001': 'Disturbance local density 2001',
#     'fea_disturbance_local_density_2002': 'Disturbance local density 2002',
#     'fea_disturbance_local_density_2003': 'Disturbance local density 2003',
#     'fea_disturbance_local_density_2004': 'Disturbance local density 2004',
#     'fea_disturbance_local_density_2005': 'Disturbance local density 2005',
#     'fea_disturbance_local_density_2006': 'Disturbance local density 2006',
#     'fea_disturbance_local_density_2007': 'Disturbance local density 2007',
#     'fea_disturbance_local_density_2008': 'Disturbance local density 2008',
#     'fea_disturbance_local_density_2009': 'Disturbance local density 2009',
#     'fea_disturbance_local_density_2010': 'Disturbance local density 2010',
#     'fea_disturbance_local_density_2011': 'Disturbance local density 2011',
#     'fea_disturbance_local_density_2012': 'Disturbance local density 2012',
#     'fea_disturbance_local_density_2013': 'Disturbance local density 2013',
#     'fea_disturbance_local_density_2014': 'Disturbance local density 2014',
#     'fea_disturbance_local_density_2015': 'Disturbance local density 2015',
#     'fea_forest_edge_distance_1990': 'Forest edge distance 1990',
#     'fea_forest_edge_distance_2000': 'Forest edge distance 2000',
#     'fea_forest_edge_distance_2010': 'Forest edge distance 2010',
#     'fea_forest_edge_distance_2011': 'Forest edge distance 2011',
#     'fea_forest_edge_distance_2012': 'Forest edge distance 2012',
#     'fea_forest_edge_distance_2013': 'Forest edge distance 2013',
#     'fea_forest_edge_distance_2014': 'Forest edge distance 2014',
#     'fea_forest_edge_distance_2015': 'Forest edge distance 2015',
#     'fea_forest_local_density_1990': 'Forest local density 1990',
#     'fea_forest_local_density_2000': 'Forest local density 2000',
#     'fea_forest_local_density_2010': 'Forest local density 2010',
#     'fea_forest_local_density_2011': 'Forest local density 2011',
#     'fea_forest_local_density_2012': 'Forest local density 2012',
#     'fea_forest_local_density_2013': 'Forest local density 2013',
#     'fea_forest_local_density_2014': 'Forest local density 2014',
#     'fea_forest_local_density_2015': 'Forest local density 2015',
#     'fea_latitude': 'Latitude',
#     'fea_longitude': 'Longitude',
#     'fea_lu_ais_edge_distance': 'Ais forest reserves with edge distance',
#     'fea_lu_berkelah_jerantut_edge_distance': 'Berkelah Jerantut Forest Reserve with edge distance',
#     'fea_lu_berkelah_kuantan_edge_distance': 'Berkelah Kuantan forest reserves with edge distance',
#     'fea_lu_berkelah_temerloh_edge_distance': 'Berkelah Temerloh Forest Reserve with edge distance',
#     'fea_lu_old-growth_protected_areas_edge_distance': 'Old-growth protected areas with edge distance',
#     'fea_lu_remen_chereh_edge_distance': 'Remen Chereh forest reserves ewith edge distance',
#     'fea_lu_tekai_tembeling_edge_distance': 'Tekai Tembeling forest reserves with edge distance',
#     'fea_lu_tekam_edge_distance': 'Tekam Forest Reserve edge distance',
#     'fea_lu_yong_edge_distance': 'Yong Lipis forest reserves with edge distance',
#     'fea_lu_yong_lipis_edge_distance': 'Yong forest reserves with edge distance',
#     'fea_topo_dsm_smooth_aspect_cosine': 'Aspect cosine smoothed',
#     'fea_topo_dsm_smooth_aspect_sine': 'Aspect sine smoothed',
#     'fea_topo_dsm_smooth_circular_variance_aspect_03': 'Circular variance aspect (3 pixel) smoothed',
#     'fea_topo_dsm_smooth_circular_variance_aspect_07': 'Circular variance aspect (7 pixel) smoothed',
#     'fea_topo_dsm_smooth_circular_variance_aspect_11': 'Circular variance aspect (11 pixel) smoothed',
#     'fea_topo_dsm_smooth_deviation_mean_elevation_03': 'Deviation mean elevation (3 pixel) smoothed',
#     'fea_topo_dsm_smooth_deviation_mean_elevation_07': 'Deviation mean elevation (7 pixel) smoothed',
#     'fea_topo_dsm_smooth_deviation_mean_elevation_11': 'Deviation mean elevation (11 pixel) smoothed',
#     'fea_topo_dsm_smooth_eastness': 'Eastness smoothed',
#     'fea_topo_dsm_smooth_elevation': 'Elevation smoothed',
#     'fea_topo_dsm_smooth_northness': 'Northness smoothed',
#     'fea_topo_dsm_smooth_profile_curvature': 'Profile curvature smoothed',
#     'fea_topo_dsm_smooth_roughness_03': 'Roughness (3 pixel) smoothed',
#     'fea_topo_dsm_smooth_roughness_07': 'Roughness (7 pixel) smoothed',
#     'fea_topo_dsm_smooth_roughness_11': 'Roughness (11 pixel) smoothed',
#     'fea_topo_dsm_smooth_slope': 'Slope smoothed',
#     'fea_topo_dsm_smooth_stream_power_index_log10': 'Stream power index log10 smoothed',
#     'fea_topo_dsm_smooth_surface_area_ratio': 'Surface area ratio smoothed',
#     'fea_topo_dsm_smooth_tangential_curvature': 'Tangential curvature smoothed',
#     'fea_topo_dsm_smooth_topographic_position_index_03': 'Topographic position index (3 pixel) smoothed',
#     'fea_topo_dsm_smooth_topographic_position_index_07': 'Topographic position index (7 pixel) smoothed',
#     'fea_topo_dsm_smooth_topographic_position_index_11': 'Topographic position index (11 pixel) smoothed',
#     'fea_topo_dsm_smooth_topographic_ruggedness_index': 'Topographic ruggedness index smoothed',
#     'fea_topo_dsm_smooth_topographic_wetness_index': 'Topographic wetness index smoothed',
#     'fea_topo_dsm_unsmooth_aspect_cosine': 'Aspect cosine unsmoothed',
#     'fea_topo_dsm_unsmooth_aspect_sine': 'Aspect sine unsmoothed',
#     'fea_topo_dsm_unsmooth_circular_variance_aspect_03': 'Circular variance aspect (3 pixel) unsmoothed',
#     'fea_topo_dsm_unsmooth_circular_variance_aspect_07': 'Circular variance aspect (7 pixel) unsmoothed',
#     'fea_topo_dsm_unsmooth_circular_variance_aspect_11': 'Circular variance aspect (11 pixel) unsmoothed',
#     'fea_topo_dsm_unsmooth_deviation_mean_elevation_03': 'Deviation mean elevation (3 pixel) unsmoothed',
#     'fea_topo_dsm_unsmooth_deviation_mean_elevation_07': 'Deviation mean elevation (7 pixel) unsmoothed',
#     'fea_topo_dsm_unsmooth_deviation_mean_elevation_11': 'Deviation mean elevation (11 pixel) unsmoothed',
#     'fea_topo_dsm_unsmooth_eastness': 'Eastness unsmoothed',
#     'fea_topo_dsm_unsmooth_elevation': 'Elevation unsmoothed',
#     'fea_topo_dsm_unsmooth_northness': 'Northness unsmoothed',
#     'fea_topo_dsm_unsmooth_profile_curvature': 'Profile curvature unsmoothed',
#     'fea_topo_dsm_unsmooth_roughness_03': 'Roughness (3 pixel) unsmoothed',
#     'fea_topo_dsm_unsmooth_roughness_07': 'Roughness (7 pixel) unsmoothed',
#     'fea_topo_dsm_unsmooth_roughness_11': 'Roughness (11 pixel) unsmoothed',
#     'fea_topo_dsm_unsmooth_slope': 'Slope unsmoothed',
#     'fea_topo_dsm_unsmooth_stream_power_index_log10': 'Stream power index log10 unsmoothed',
#     'fea_topo_dsm_unsmooth_surface_area_ratio': 'Surface area ratio unsmoothed',
#     'fea_topo_dsm_unsmooth_tangential_curvature': 'Tangential curvature unsmoothed',
#     'fea_topo_dsm_unsmooth_topographic_position_index_03': 'Topographic position index (3 pixel) unsmoothed',
#     'fea_topo_dsm_unsmooth_topographic_position_index_07': 'Topographic position index (7 pixel) unsmoothed',
#     'fea_topo_dsm_unsmooth_topographic_position_index_11': 'Topographic position index (11 pixel) unsmoothed',
#     'fea_topo_dsm_unsmooth_topographic_ruggedness_index': 'Topographic ruggedness index unsmoothed',
#     'fea_topo_dsm_unsmooth_topographic_wetness_index': 'Topographic wetness index unsmoothed',
#     'fea_beam': 'GEDI beam',
#     'fea_sensitivity': 'GEDI sensitivity',
# }

In [ ]:
# AGBD

# Number of features: 131

feature_name_mapping = {
    'fea_coast_proximity_km': 'Coast proximity (km)',
    'fea_disturbance_edge_distance_1995': 'Disturbance with edge distance 1995',
    'fea_disturbance_edge_distance_1996': 'Disturbance with edge distance 1996',
    'fea_disturbance_edge_distance_1997': 'Disturbance with edge distance 1997',
    'fea_disturbance_edge_distance_1998': 'Disturbance with edge distance 1998',
    'fea_disturbance_edge_distance_1999': 'Disturbance with edge distance 1999',
    'fea_disturbance_edge_distance_2000': 'Disturbance with edge distance 2000',
    'fea_disturbance_edge_distance_2001': 'Disturbance with edge distance 2001',
    'fea_disturbance_edge_distance_2002': 'Disturbance with edge distance 2002',
    'fea_disturbance_edge_distance_2003': 'Disturbance with edge distance 2003',
    'fea_disturbance_edge_distance_2004': 'Disturbance with edge distance 2004',
    'fea_disturbance_edge_distance_2005': 'Disturbance with edge distance 2005',
    'fea_disturbance_edge_distance_2006': 'Disturbance with edge distance 2006',
    'fea_disturbance_edge_distance_2007': 'Disturbance with edge distance 2007',
    'fea_disturbance_edge_distance_2008': 'Disturbance with edge distance 2008',
    'fea_disturbance_edge_distance_2009': 'Disturbance with edge distance 2009',
    'fea_disturbance_edge_distance_2010': 'Disturbance with edge distance 2010',
    'fea_disturbance_edge_distance_2011': 'Disturbance with edge distance 2011',
    'fea_disturbance_edge_distance_2012': 'Disturbance with edge distance 2012',
    'fea_disturbance_edge_distance_2013': 'Disturbance with edge distance 2013',
    'fea_disturbance_edge_distance_2014': 'Disturbance with edge distance 2014',
    'fea_disturbance_edge_distance_2015': 'Disturbance with edge distance 2015',
    'fea_disturbance_edge_distance_2016': 'Disturbance with edge distance 2016',
    'fea_disturbance_edge_distance_2017': 'Disturbance with edge distance 2017',
    'fea_disturbance_edge_distance_2018': 'Disturbance with edge distance 2018',
    'fea_disturbance_edge_distance_2019': 'Disturbance with edge distance 2019',
    'fea_disturbance_edge_distance_2020': 'Disturbance with edge distance 2020',
    'fea_disturbance_edge_distance_2021': 'Disturbance with edge distance 2021',
    'fea_disturbance_edge_distance_2022': 'Disturbance with edge distance 2022',
    'fea_disturbance_edge_distance_2023': 'Disturbance with edge distance 2023',
    'fea_disturbance_local_density_1995': 'Disturbance local density 1995',
    'fea_disturbance_local_density_1996': 'Disturbance local density 1996',
    'fea_disturbance_local_density_1997': 'Disturbance local density 1997',
    'fea_disturbance_local_density_1998': 'Disturbance local density 1998',
    'fea_disturbance_local_density_1999': 'Disturbance local density 1999',
    'fea_disturbance_local_density_2000': 'Disturbance local density 2000',
    'fea_disturbance_local_density_2001': 'Disturbance local density 2001',
    'fea_disturbance_local_density_2002': 'Disturbance local density 2002',
    'fea_disturbance_local_density_2003': 'Disturbance local density 2003',
    'fea_disturbance_local_density_2004': 'Disturbance local density 2004',
    'fea_disturbance_local_density_2005': 'Disturbance local density 2005',
    'fea_disturbance_local_density_2006': 'Disturbance local density 2006',
    'fea_disturbance_local_density_2007': 'Disturbance local density 2007',
    'fea_disturbance_local_density_2008': 'Disturbance local density 2008',
    'fea_disturbance_local_density_2009': 'Disturbance local density 2009',
    'fea_disturbance_local_density_2010': 'Disturbance local density 2010',
    'fea_disturbance_local_density_2011': 'Disturbance local density 2011',
    'fea_disturbance_local_density_2012': 'Disturbance local density 2012',
    'fea_disturbance_local_density_2013': 'Disturbance local density 2013',
    'fea_disturbance_local_density_2014': 'Disturbance local density 2014',
    'fea_disturbance_local_density_2015': 'Disturbance local density 2015',
    'fea_disturbance_local_density_2016': 'Disturbance local density 2016',
    'fea_disturbance_local_density_2017': 'Disturbance local density 2017',
    'fea_disturbance_local_density_2018': 'Disturbance local density 2018',
    'fea_disturbance_local_density_2019': 'Disturbance local density 2019',
    'fea_disturbance_local_density_2020': 'Disturbance local density 2020',
    'fea_disturbance_local_density_2021': 'Disturbance local density 2021',
    'fea_disturbance_local_density_2022': 'Disturbance local density 2022',
    'fea_disturbance_local_density_2023': 'Disturbance local density 2023',
    'fea_forest_edge_distance_1995': 'Forest edge distance 1995',
    'fea_forest_edge_distance_2008': 'Forest edge distance 2008',
    'fea_forest_edge_distance_2021': 'Forest edge distance 2021',
    'fea_forest_edge_distance_2022': 'Forest edge distance 2022',
    'fea_forest_edge_distance_2023': 'Forest edge distance 2023',
    'fea_forest_local_density_1995': 'Forest local density 1995',
    'fea_forest_local_density_2008': 'Forest local density 2008',
    'fea_forest_local_density_2021': 'Forest local density 2021',
    'fea_forest_local_density_2022': 'Forest local density 2022',
    'fea_forest_local_density_2023': 'Forest local density 2023',
    'fea_latitude': 'Latitude',
    'fea_longitude': 'Longitude',
    'fea_lu_ais_edge_distance': 'Ais forest reserves with edge distance',
    'fea_lu_berkelah_jerantut_edge_distance': 'Berkelah Jerantut Forest Reserve with edge distance',
    'fea_lu_berkelah_kuantan_edge_distance': 'Berkelah Kuantan forest reserves with edge distance',
    'fea_lu_berkelah_temerloh_edge_distance': 'Berkelah Temerloh Forest Reserve with edge distance',
    'fea_lu_old-growth_protected_areas_edge_distance': 'Old-growth protected areas with edge distance',
    'fea_lu_remen_chereh_edge_distance': 'Remen Chereh forest reserves ewith edge distance',
    'fea_lu_tekai_tembeling_edge_distance': 'Tekai Tembeling forest reserves with edge distance',
    'fea_lu_tekam_edge_distance': 'Tekam Forest Reserve edge distance',
    'fea_lu_yong_edge_distance': 'Yong Lipis forest reserves with edge distance',
    'fea_lu_yong_lipis_edge_distance': 'Yong forest reserves with edge distance',
    'fea_topo_dtm_smooth_aspect_cosine': 'Aspect cosine smoothed',
    'fea_topo_dtm_smooth_aspect_sine': 'Aspect sine smoothed',
    'fea_topo_dtm_smooth_circular_variance_aspect_03': 'Circular variance aspect (3 pixel) smoothed',
    'fea_topo_dtm_smooth_circular_variance_aspect_07': 'Circular variance aspect (7 pixel) smoothed',
    'fea_topo_dtm_smooth_circular_variance_aspect_11': 'Circular variance aspect (11 pixel) smoothed',
    'fea_topo_dtm_smooth_deviation_mean_elevation_03': 'Deviation mean elevation (3 pixel) smoothed',
    'fea_topo_dtm_smooth_deviation_mean_elevation_07': 'Deviation mean elevation (7 pixel) smoothed',
    'fea_topo_dtm_smooth_deviation_mean_elevation_11': 'Deviation mean elevation (11 pixel) smoothed',
    'fea_topo_dtm_smooth_eastness': 'Eastness smoothed',
    'fea_topo_dtm_smooth_elevation': 'Elevation smoothed',
    'fea_topo_dtm_smooth_northness': 'Northness smoothed',
    'fea_topo_dtm_smooth_profile_curvature': 'Profile curvature smoothed',
    'fea_topo_dtm_smooth_roughness_03': 'Roughness (3 pixel) smoothed',
    'fea_topo_dtm_smooth_roughness_07': 'Roughness (7 pixel) smoothed',
    'fea_topo_dtm_smooth_roughness_11': 'Roughness (11 pixel) smoothed',
    'fea_topo_dtm_smooth_slope': 'Slope smoothed',
    'fea_topo_dtm_smooth_stream_power_index_log10': 'Stream power index log10 smoothed',
    'fea_topo_dtm_smooth_surface_area_ratio': 'Surface area ratio smoothed',
    'fea_topo_dtm_smooth_tangential_curvature': 'Tangential curvature smoothed',
    'fea_topo_dtm_smooth_topographic_position_index_03': 'Topographic position index (3 pixel) smoothed',
    'fea_topo_dtm_smooth_topographic_position_index_07': 'Topographic position index (7 pixel) smoothed',
    'fea_topo_dtm_smooth_topographic_position_index_11': 'Topographic position index (11 pixel) smoothed',
    'fea_topo_dtm_smooth_topographic_ruggedness_index': 'Topographic ruggedness index smoothed',
    'fea_topo_dtm_smooth_topographic_wetness_index': 'Topographic wetness index smoothed',
    'fea_topo_dtm_unsmooth_aspect_cosine': 'Aspect cosine unsmoothed',
    'fea_topo_dtm_unsmooth_aspect_sine': 'Aspect sine unsmoothed',
    'fea_topo_dtm_unsmooth_circular_variance_aspect_03': 'Circular variance aspect (3 pixel) unsmoothed',
    'fea_topo_dtm_unsmooth_circular_variance_aspect_07': 'Circular variance aspect (7 pixel) unsmoothed',
    'fea_topo_dtm_unsmooth_circular_variance_aspect_11': 'Circular variance aspect (11 pixel) unsmoothed',
    'fea_topo_dtm_unsmooth_deviation_mean_elevation_03': 'Deviation mean elevation (3 pixel) unsmoothed',
    'fea_topo_dtm_unsmooth_deviation_mean_elevation_07': 'Deviation mean elevation (7 pixel) unsmoothed',
    'fea_topo_dtm_unsmooth_deviation_mean_elevation_11': 'Deviation mean elevation (11 pixel) unsmoothed',
    'fea_topo_dtm_unsmooth_eastness': 'Eastness unsmoothed',
    'fea_topo_dtm_unsmooth_elevation': 'Elevation unsmoothed',
    'fea_topo_dtm_unsmooth_northness': 'Northness unsmoothed',
    'fea_topo_dtm_unsmooth_profile_curvature': 'Profile curvature unsmoothed',
    'fea_topo_dtm_unsmooth_roughness_03': 'Roughness (3 pixel) unsmoothed',
    'fea_topo_dtm_unsmooth_roughness_07': 'Roughness (7 pixel) unsmoothed',
    'fea_topo_dtm_unsmooth_roughness_11': 'Roughness (11 pixel) unsmoothed',
    'fea_topo_dtm_unsmooth_slope': 'Slope unsmoothed',
    'fea_topo_dtm_unsmooth_stream_power_index_log10': 'Stream power index log10 unsmoothed',
    'fea_topo_dtm_unsmooth_surface_area_ratio': 'Surface area ratio unsmoothed',
    'fea_topo_dtm_unsmooth_tangential_curvature': 'Tangential curvature unsmoothed',
    'fea_topo_dtm_unsmooth_topographic_position_index_03': 'Topographic position index (3 pixel) unsmoothed',
    'fea_topo_dtm_unsmooth_topographic_position_index_07': 'Topographic position index (7 pixel) unsmoothed',
    'fea_topo_dtm_unsmooth_topographic_position_index_11': 'Topographic position index (11 pixel) unsmoothed',
    'fea_topo_dtm_unsmooth_topographic_ruggedness_index': 'Topographic ruggedness index unsmoothed',
    'fea_topo_dtm_unsmooth_topographic_wetness_index': 'Topographic wetness index unsmoothed',
    'fea_beam': 'GEDI beam',
    'fea_sensitivity': 'GEDI sensitivity',
}

In [ ]:
# Sample model dataset to reduce computation time
# If False, sample by number instead of percent
sample_model_dataset_by_percent, sample_model_dataset_value = True, 100

# Show how each feature's value affects its impact on predictions (with trend lines)
generate_feature_effects = True

# Polynomial degree for feature effect trendline
poly_degree = 4

# Generate full summary plot (time consuming, will generate only no_top_features if False)
generate_summary_full_plot = False

# Remove feature value outliers for plotting visualisation only (does not affect calculated curves)
remove_outliers = True
plot_feature_percentile_range = 99.9  # Removes 0.05% from each tail for visualisation

# Number of top (most important) features to plot
no_top_features = 20

# Set non-interactive backend for better performance
matplotlib.use('Agg')

# Verify model and create explainer
assert os.path.exists(model_description_dir) and os.path.exists(final_model_dir), "Model files must exist to continue."

# Load model appropriately for SHAP
if categorise_target:
    # For classification, recreate the sklearn model for better SHAP compatibility
    with open(model_description_dir) as f: model_desc = json.load(f)
    final_params = ast.literal_eval(model_desc["hyperparameters"])
    # Ensure num_class is available for multiclass SHAP
    if categorise_target and multiclass: final_params["num_class"] = num_class
    model = XGBPredictor(**final_params)
    model.load_model(final_model_dir)
    explainer = shap.TreeExplainer(model)
else:
    # For regression, use booster directly
    model = xgb.Booster()
    model.load_model(final_model_dir)
    explainer = shap.TreeExplainer(model)
print("Model loaded and SHAP explainer created.")

# Set up feature naming
try: feature_name_mapping
except NameError:
    feature_name_mapping = {feature: feature for feature in selected_features}
    print("Using default feature names.")

# Load or prepare dataset
processed_data_path = join(shap_cache_dir, 'shap_processed_data.pkl')
if exists(processed_data_path):
    with open(processed_data_path, 'rb') as f:
        X, y = pickle.load(f)
    print("Processed data loaded from cache.")
else:
    # Sample and prepare dataset
    model_dataset_shap = (model_dataset.sample(frac=sample_model_dataset_value/100, random_state=1) if sample_model_dataset_by_percent
                        else model_dataset.sample(n=sample_model_dataset_value, random_state=1))
    X = model_dataset_shap[selected_features].copy()
    y = model_dataset_shap[selected_target]

    # Handle categorical features for SHAP
    for col in X.select_dtypes(include=['category']).columns: X[col] = X[col].cat.codes
    # Ensure data types are compatible with SHAP
    if categorise_target: X = X.astype('float32')

    # Clean data
    mask = ~X.isin([np.nan, np.inf, -np.inf]).any(axis=1) & ~y.isin([np.nan, np.inf, -np.inf])
    X, y = X[mask], y[mask]

    with open(processed_data_path, 'wb') as f:
        pickle.dump((X, y), f)
    print("Data processed and cached.")

# Compute or load SHAP values
shap_values_path = join(shap_cache_dir, 'shap_values.pkl')
if exists(shap_values_path):
    with open(shap_values_path, 'rb') as f:
        shap_values = pickle.load(f)
    print("SHAP values loaded from cache.")
else:
    # For classification, use the explainer method that works with sklearn models
    if categorise_target:
        shap_values = explainer(X, check_additivity=False)
        # Extract values if it's an Explanation object
        if hasattr(shap_values, 'values'): shap_values = shap_values.values
    # For regression, use the traditional method
    else: shap_values = explainer.shap_values(X, check_additivity=False)
    with open(shap_values_path, 'wb') as f: pickle.dump(shap_values, f)
    print("SHAP values computed and cached.")

# Initial data prep
X_renamed = X.rename(columns=feature_name_mapping).reset_index(drop=True)

# Handle multiclass SHAP values (3D array: samples × features × classes)
if shap_values.ndim == 3:
    # For multiclass, take mean across classes to get overall feature importance
    shap_values_2d = np.mean(shap_values, axis=2)
    print(f"Multiclass SHAP values aggregated across {shap_values.shape[2]} classes")
# For binary classification or regression, use as-is
else: shap_values_2d = shap_values

shap_values_df = pd.DataFrame(shap_values_2d, columns=X_renamed.columns)
y = y.reset_index(drop=True)

# Update shap_values for later plotting functions
shap_values = shap_values_2d

# Remove zero variance features
zero_var_features = X_renamed.columns[X_renamed.nunique() <= 1]
if not zero_var_features.empty:
    print(f"Excluding zero variance features: {zero_var_features.tolist()}")
    X_renamed = X_renamed.drop(columns=zero_var_features)
    shap_values_df = shap_values_df.drop(columns=zero_var_features)
    shap_values = shap_values_df.values

# Clean data for plotting
mask = ~X_renamed.replace([np.inf, -np.inf], np.nan).isna().any(axis=1)
X_renamed = X_renamed[mask]
shap_values_df = shap_values_df[mask]
y = y[mask]
shap_values = shap_values_df.values

if X_renamed.empty or shap_values_df.empty:
    print("No data available for plotting after cleaning.")
    exit()

# Define output paths
shap_settings_path = join(shap_dir, 'shap_settings.csv')
feature_analysis_path = join(shap_dir, 'shap_feature_analysis.csv')
feature_importance_path = join(shap_dir, 'shap_feature_importance.csv')
summary_plot_top_path = join(shap_dir, 'shap_summary_plot_top')
summary_plot_full_path = join(shap_dir, 'shap_summary_plot_full')
feature_importance_top_path = join(shap_dir, 'shap_feature_importance_top')
feature_importance_full_path = join(shap_dir, 'shap_feature_importance_full')

# Save SHAP analysis settings
if not exists(shap_settings_path):
    settings_df = pd.DataFrame({
        'Setting': [
            'polynomial_degree',
            'remove_outliers',
            'plot_feature_percentile_range',
            'sample_model_dataset_by_percent',
            'sample_model_dataset_value',
            'no_top_features'
        ],
        'Value': [
            poly_degree,
            remove_outliers,
            plot_feature_percentile_range,
            sample_model_dataset_by_percent,
            sample_model_dataset_value,
            no_top_features
        ]
    })
    settings_df.to_csv(shap_settings_path, index=False)
    print("SHAP settings saved.")

# Calculate feature importance
importance_df = pd.DataFrame({'Feature': X_renamed.columns, 'Mean Absolute SHAP Value': np.abs(shap_values_df).mean(axis=0)})
importance_df.sort_values('Mean Absolute SHAP Value', ascending=False, inplace=True)
importance_df.to_csv(feature_importance_path, index=False)

# Calculate feature statistics
reverse_name_mapping = {v: k for k, v in feature_name_mapping.items()}
if not exists(feature_analysis_path):
    warnings.filterwarnings('ignore', message='Polyfit may be poorly conditioned')

    # Pre-extract data
    X_values = X_renamed.values
    shap_values_array = shap_values_df.values

    # Calculate relationship statistics and neutral effect values for each feature
    feature_stats = []
    for feature in X_renamed.columns:
        feature_idx = X_renamed.columns.get_loc(feature)
        x = X_values[:, feature_idx]
        y_shap = shap_values_array[:, feature_idx]

        x_mean, x_scale = np.mean(x), np.std(x)

        # Normalize for stable fitting
        if x_scale != 0:
            x_norm = (x - x_mean) / x_scale

            # Linear fit
            lin_coef = np.polyfit(x_norm, y_shap, 1)
            lin_pred = np.polyval(lin_coef, x_norm)
            ss_tot = np.sum((y_shap - np.mean(y_shap))**2)
            lin_r2 = 1 - (np.sum((y_shap - lin_pred)**2) / ss_tot) if ss_tot > 0 else 0.0

            # Polynomial fit
            poly_coef = np.polyfit(x_norm, y_shap, poly_degree)
            poly_pred = np.polyval(poly_coef, x_norm)
            poly_r2 = 1 - (np.sum((y_shap - poly_pred)**2) / ss_tot) if ss_tot > 0 else 0.0

            # Determine predominant direction based on percentage of positive effects
            pct_positive = (y_shap > 0).sum() / len(y_shap) * 100
            direction = "Positive" if pct_positive > 70 else "Negative" if pct_positive < 30 else "Mixed"

            # Calculate neutral effect value: feature value where mean SHAP impact is minimal
            # Used for setting covariate features to constant values during spatial predictions
            # e.g. when creating raster outputs where certain features must be held constant
            # Binning averages SHAP values locally to find robust neutral point
            # Single-sample approach unreliable due to outliers and feature interactions
            n_unique = len(np.unique(x))
            if n_unique <= 20:
                # Categorical or low-cardinality: bin by unique value directly
                unique_vals = np.unique(x)
                bin_means = unique_vals.astype(float)
                bin_shaps = [y_shap[x == v].mean() for v in unique_vals]
                n_bins = n_unique
            else:
                # Continuous: quantile-based binning with adaptive bin count (sqrt of n)
                # Equal sample distribution per bin prevents unreliable estimates in tails
                n_bins = int(np.sqrt(len(x)))
                bin_edges = np.percentile(x, np.linspace(0, 100, n_bins + 1))
                bin_indices = np.clip(np.digitize(x, bin_edges), 1, n_bins)

                bin_means, bin_shaps = [], []
                for i in range(1, n_bins + 1):
                    mask = bin_indices == i
                    if mask.sum() > 0:
                        bin_means.append(x[mask].mean())
                        bin_shaps.append(y_shap[mask].mean())

            # Find bin where mean SHAP closest to zero
            if len(bin_shaps) > 0:
                neutral_idx = np.argmin(np.abs(bin_shaps))
                neutral_value = bin_means[neutral_idx]
                neutral_shap = bin_shaps[neutral_idx]
            else:
                neutral_value, neutral_shap = np.median(x), 0.0
        else:
            # Zero variance feature
            lin_coef, poly_coef = np.array([0, 0]), np.zeros(poly_degree + 1)
            lin_r2, poly_r2 = 0.0, 0.0
            direction = "Mixed"
            neutral_value, neutral_shap = x_mean, 0.0
            n_bins = 0

        feature_stats.append({
            'Feature': feature,
            'Dataset name': reverse_name_mapping.get(feature, feature),
            'Feature_Mean': x_mean,
            'Feature_Std': x_scale,
            'Linear_Coefficient': lin_coef[0],
            'Linear_R2': lin_r2,
            'Polynomial_Coefficients': ','.join(map(str, poly_coef)),
            'Polynomial_R2': poly_r2,
            'Predominant_Direction': direction,
            'Neutral_Effect_Value': neutral_value,
            'SHAP_at_Neutral': neutral_shap,
            'Neutral_Effect_Bins': n_bins
        })

    # Save feature statistics sorted alphabetically by feature name
    stats_df = pd.DataFrame(feature_stats)
    stats_df.sort_values('Feature', inplace=True)
    stats_df.to_csv(feature_analysis_path, index=False)
    print("Feature relationship statistics and neutral effect values generated.")
else:
    print("Feature statistics already exist.")

# Generate top feature importance plot
if not exists(f"{feature_importance_top_path}.png") or not exists(f"{feature_importance_top_path}.svg"):
   shap.summary_plot(shap_values, X_renamed, plot_type="bar", show=False, max_display=no_top_features)
   plt.gcf().set_size_inches(12, 0.4 * no_top_features)
   plt.tight_layout()
   if not exists(f"{feature_importance_top_path}.png"):
       plt.savefig(f"{feature_importance_top_path}.png", bbox_inches='tight', dpi=150, pad_inches=0.1)
   if not exists(f"{feature_importance_top_path}.svg"):
       plt.savefig(f"{feature_importance_top_path}.svg", bbox_inches='tight', dpi=150, pad_inches=0.1, format='svg')
   plt.close()
   print("Top feature importance plot created.")
else: print("Top feature importance plot already exists.")

# Generate full feature importance plot
if not exists(f"{feature_importance_full_path}.png") or not exists(f"{feature_importance_full_path}.svg"):
   shap.summary_plot(shap_values, X_renamed, plot_type="bar", show=False, max_display=len(X_renamed.columns))
   plt.gcf().set_size_inches(12, 0.4 * len(X_renamed.columns))
   plt.tight_layout()
   if not exists(f"{feature_importance_full_path}.png"):
       plt.savefig(f"{feature_importance_full_path}.png", bbox_inches='tight', dpi=150, pad_inches=0.1)
   if not exists(f"{feature_importance_full_path}.svg"):
       plt.savefig(f"{feature_importance_full_path}.svg", bbox_inches='tight', dpi=150, pad_inches=0.1, format='svg')
   plt.close()
   print("Full feature importance plot created.")
else: print("Full feature importance plot already exists.")

# Generate top summary plot
if not exists(f"{summary_plot_top_path}.png") or not exists(f"{summary_plot_top_path}.svg"):
   shap.summary_plot(shap_values, X_renamed, show=False, max_display=no_top_features)
   plt.gcf().set_size_inches(12, 0.4 * no_top_features)
   plt.tight_layout()
   if not exists(f"{summary_plot_top_path}.png"):
       plt.savefig(f"{summary_plot_top_path}.png", bbox_inches='tight', dpi=150, pad_inches=0.1)
   if not exists(f"{summary_plot_top_path}.svg"):
       plt.savefig(f"{summary_plot_top_path}.svg", bbox_inches='tight', dpi=150, pad_inches=0.1, format='svg')
   plt.close()
   print("Top summary plot created.")
else: print("Top summary plot already exists.")

# Generate full summary plot
if generate_summary_full_plot:
  if not exists(f"{summary_plot_full_path}.png") or not exists(f"{summary_plot_full_path}.svg"):
    shap.summary_plot(shap_values, X_renamed, show=False, max_display=len(X_renamed.columns))
    plt.gcf().set_size_inches(12, 0.4 * len(X_renamed.columns))
    plt.tight_layout()
    if not exists(f"{summary_plot_full_path}.png"):
        plt.savefig(f"{summary_plot_full_path}.png", bbox_inches='tight', dpi=150, pad_inches=0.1)
    if not exists(f"{summary_plot_full_path}.svg"):
        plt.savefig(f"{summary_plot_full_path}.svg", bbox_inches='tight', dpi=150, pad_inches=0.1, format='svg')
    plt.close()
    print("Full summary plot created.")
  else: print("Full summary plot already exists.")

# Generate feature effect plots using statistics from feature analysis CSV
if generate_feature_effects:
   # Load feature statistics containing coefficients and neutral effect values
   feature_stats_df = pd.read_csv(feature_analysis_path)

   # Pre-extract data for plotting
   X_values = X_renamed.values
   shap_values_array = shap_values_df.values
   feature_names = importance_df['Feature'].tolist()

   for feature in feature_names:
       feature_effect_png_path = join(shap_feature_effect_plots_dir, f'feature_effect_{re.sub(r'[<>:"/\\|?*]', '_', feature)}.png')
       feature_effect_svg_path = join(shap_feature_effect_plots_dir, f'feature_effect_{re.sub(r'[<>:"/\\|?*]', '_', feature)}.svg')
       if not exists(feature_effect_png_path) or not exists(feature_effect_svg_path):
          # Get feature data
          feature_idx = X_renamed.columns.get_loc(feature)
          feature_values = X_values[:, feature_idx]
          shap_values_feature = shap_values_array[:, feature_idx]

          # Get feature statistics for trend lines and neutral effect value from alphabetically sorted CSV
          feature_row = feature_stats_df[feature_stats_df['Feature'] == feature].iloc[0]
          x_mean = feature_row['Feature_Mean']
          x_scale = feature_row['Feature_Std']
          neutral_value = feature_row['Neutral_Effect_Value']

          # Parse polynomial coefficients from CSV
          poly_coef = np.array([float(c) for c in feature_row['Polynomial_Coefficients'].split(',')])
          lin_coef = np.array([feature_row['Linear_Coefficient'], 0])

          # Determine x range for plotting with optional outlier removal for visualisation
          if remove_outliers:
              lower = np.percentile(feature_values, (100 - plot_feature_percentile_range) / 2)
              upper = np.percentile(feature_values, 100 - (100 - plot_feature_percentile_range) / 2)
              mask = (feature_values >= lower) & (feature_values <= upper)
              feature_values_plot = feature_values[mask]
              shap_values_plot = shap_values_feature[mask]
              x_range = np.linspace(lower, upper, 200)
          else:
              feature_values_plot = feature_values
              shap_values_plot = shap_values_feature
              x_range = np.linspace(feature_values.min(), feature_values.max(), 200)

          fig, ax = plt.subplots(figsize=(10, 6))
          ax.scatter(feature_values_plot, shap_values_plot, alpha=0.5, rasterized=True)

          # Draw trend lines using coefficients from statistics CSV
          if x_scale != 0:
              x_range_norm = (x_range - x_mean) / x_scale

              # Linear trend
              lin_pred = np.polyval(lin_coef, x_range_norm)
              ax.plot(x_range, lin_pred, "--", color='black', alpha=0.8, label='Linear trend')

              # Polynomial trend
              poly_pred = np.polyval(poly_coef, x_range_norm)
              ax.plot(x_range, poly_pred, "--", color='red', alpha=0.8, label=f'Polynomial trend (degree {poly_degree})')

          # Add neutral effect line from statistics CSV
          ax.axvline(x=neutral_value, color='blue', linestyle=':', linewidth=2, alpha=0.8, label='Neutral effect value')

          ax.set_title(f'SHAP Feature Effect: {feature}')
          ax.set_xlabel(f'{feature} value')
          ax.set_ylabel('SHAP value (impact on prediction)')
          ax.legend(loc='upper right')
          if not exists(feature_effect_png_path):
            plt.savefig(feature_effect_png_path, bbox_inches='tight', dpi=150, pad_inches=0.1)
          if not exists(feature_effect_svg_path):
            plt.savefig(feature_effect_svg_path, bbox_inches='tight', dpi=150, pad_inches=0.1, format='svg')
          plt.close()
   print("Feature effect plots generated.")

print("Analysis complete. All plots and statistics have been generated.")

## Combine features

In [ ]:
# Modify these rules to control feature combination behavior
# Run after the main SHAP calculations (uses the cache)

# Automatic smoothed/unsmoothed detection
# Detects and combines features with both smoothed and unsmoothed versions
shap_smoothed_combined_dir = join(model_dataset_dir, 'shap_smoothed_combined')
combine_smoothed_unsmoothed = True
generate_smoothed_importance_plots = True  # bar plots showing feature importance
generate_smoothed_summary_plots = True     # scatter plots showing feature effects
generate_smoothed_summary_full_plot = False # Time consuming, will generate only no_top_features if False
generate_smoothed_effect_plots = True      # individual feature effect plots with trend lines

# Conceptual combinations
# Sums SHAP values for features containing specified patterns
generate_conceptual_combinations = True

# Each combination creates a numbered directory with saved grouping rules
conceptual_combinations_list = [
    {
        'Topographic elevation': ['elevation'],
        'Topographic slope': ['slope'],
        'Topographic surface area ratio': ['surface_area_ratio'],
        'Topographic orientation': ['aspect_cosine', 'aspect_sine', 'northness', 'eastness'],
        'Topographic shape': ['profile_curvature', 'tangential_curvature', 'topographic_position_index'],
        'Topographic complexity': ['roughness', 'topographic_ruggedness_index', 'circular_variance_aspect', 'deviation_mean_elevation'],
        'Hydrography': ['stream_power_index', 'topographic_wetness_index'],
        'Disturbance with edge effects': ['disturbance_edge_distance', 'disturbance_local_density'],
        'Forest edge effects': ['forest_edge_distance', 'forest_local_density'],
        'Old-growth': ['old-growth'],
        'Forest reserve management': ['yong', 'tekai', 'tekam', 'remen', 'berkelah', 'ais'],
        'Geographic location': ['coast', 'latitude', 'longitude'],
        'GEDI parameters': ['beam', 'sensitivity'],
    }
]

In [ ]:
# Validate conceptual combinations
for combo_idx, conceptual_combinations in enumerate(conceptual_combinations_list, 1):
    print(f"Validating combination set {combo_idx}:")

    all_original_features = list(feature_name_mapping.keys())
    available_features = list(feature_name_mapping.values())

    covered_features = set()
    unmatched_patterns = []

    for concept_name, patterns in conceptual_combinations.items():
        concept_matches = set()
        for pattern in patterns:
            pattern_matches = [orig_name for orig_name in all_original_features if pattern in orig_name]
            if not pattern_matches:
                unmatched_patterns.append((concept_name, pattern))
            else:
                renamed_matches = [feature_name_mapping[orig] for orig in pattern_matches
                                 if feature_name_mapping[orig] in available_features]
                concept_matches.update(renamed_matches)
        covered_features.update(concept_matches)
        print(f"  {concept_name}: {len(concept_matches)} features")

    # Report unmatched patterns
    if unmatched_patterns:
        print(f"  Unmatched patterns: {len(unmatched_patterns)}")
        for concept_name, pattern in unmatched_patterns:
            print(f"    {concept_name} -> {pattern}")

    # Report uncovered features
    uncovered_features = set(available_features) - covered_features
    if uncovered_features:
        print(f"  Uncovered features: {len(uncovered_features)}")
        for feature in sorted(uncovered_features):
            print(f"    {feature}")

    # Coverage summary
    coverage_pct = (len(covered_features) / len(available_features)) * 100 if available_features else 0
    print(f"  Coverage: {len(covered_features)}/{len(available_features)} features ({coverage_pct:.1f}%)")

In [ ]:
# Polynomial degree for feature effect trendline
poly_degree = 4

# Remove feature value outliers for plotting visualisation only (does not affect calculated curves)
remove_outliers = True
plot_feature_percentile_range = 99.9  # Removes 0.05% from each tail for visualisation

# Number of top (most important) features to plot
no_top_features = 20

# Set non-interactive backend for better performance
matplotlib.use('Agg')

# Define cache paths
shap_dir = join(model_dataset_dir, 'shap')
shap_cache_dir = join(shap_dir, 'shap_cache')
processed_data_path = join(shap_cache_dir, 'shap_processed_data.pkl')
shap_values_path = join(shap_cache_dir, 'shap_values.pkl')

# Create directories for combined analyses
os.makedirs(shap_smoothed_combined_dir, exist_ok=True)

# Load cached data - no recomputation required
with open(processed_data_path, 'rb') as f:
    X_original, y_original = pickle.load(f)
with open(shap_values_path, 'rb') as f:
    shap_values_original = pickle.load(f)

print("Cached data loaded for feature combination analysis.")

# Handle multiclass SHAP values for combination analysis
if shap_values_original.ndim == 3:
    shap_values_2d_original = np.mean(shap_values_original, axis=2)
else:
    shap_values_2d_original = shap_values_original

# Apply feature name mapping and prepare base datasets
X_renamed_original = X_original.rename(columns=feature_name_mapping).reset_index(drop=True)
shap_values_df_original = pd.DataFrame(shap_values_2d_original, columns=X_renamed_original.columns)
y_reset = y_original.reset_index(drop=True)

# Remove zero variance features
zero_var_features = X_renamed_original.columns[X_renamed_original.nunique() <= 1]
if not zero_var_features.empty:
    X_renamed_original = X_renamed_original.drop(columns=zero_var_features)
    shap_values_df_original = shap_values_df_original.drop(columns=zero_var_features)

# Clean data for combination analysis
mask = ~X_renamed_original.replace([np.inf, -np.inf], np.nan).isna().any(axis=1)
X_clean = X_renamed_original[mask].reset_index(drop=True)
shap_clean = shap_values_df_original[mask].reset_index(drop=True)
y_clean = y_reset[mask].reset_index(drop=True)

if combine_smoothed_unsmoothed:
    print("Processing smoothed/unsmoothed feature combinations...")

    # Detect smoothed/unsmoothed pairs automatically by removing smoothing indicators
    feature_base_names = {}
    for feature in X_clean.columns:
        base_name = feature
        is_smoothed = base_name.endswith(' smoothed')
        is_unsmoothed = base_name.endswith(' unsmoothed')

        if is_smoothed:
            base_name = base_name.replace(' smoothed', '')
        elif is_unsmoothed:
            base_name = base_name.replace(' unsmoothed', '')
        else:
            continue  # Skip features without smoothing indicators

        if base_name not in feature_base_names:
            feature_base_names[base_name] = {}

        if is_smoothed:
            feature_base_names[base_name]['smoothed'] = feature
        elif is_unsmoothed:
            feature_base_names[base_name]['unsmoothed'] = feature

    # Identify complete pairs with both smoothed and unsmoothed versions
    smoothed_unsmoothed_pairs = {base: features for base, features in feature_base_names.items()
                                if 'smoothed' in features and 'unsmoothed' in features}

    print(f"Found {len(smoothed_unsmoothed_pairs)} smoothed/unsmoothed pairs.")

    # Create combined datasets by averaging feature values and summing SHAP values
    X_smoothed_combined = X_clean.copy()
    shap_smoothed_combined = shap_clean.copy()

    # Add combined features and remove individual smoothed/unsmoothed features
    features_to_remove = []
    for base_name, feature_pair in smoothed_unsmoothed_pairs.items():
        smoothed_feature = feature_pair['smoothed']
        unsmoothed_feature = feature_pair['unsmoothed']

        # Average feature values - best estimate of underlying concept
        X_smoothed_combined[base_name] = (X_clean[smoothed_feature] + X_clean[unsmoothed_feature]) / 2

        # Sum SHAP values - total contribution of concept to predictions
        shap_smoothed_combined[base_name] = shap_clean[smoothed_feature] + shap_clean[unsmoothed_feature]

        features_to_remove.extend([smoothed_feature, unsmoothed_feature])

    # Remove original smoothed/unsmoothed features
    X_smoothed_combined = X_smoothed_combined.drop(columns=features_to_remove)
    shap_smoothed_combined = shap_smoothed_combined.drop(columns=features_to_remove)

    # Calculate importance for combined features
    importance_smoothed_df = pd.DataFrame({
        'Feature': X_smoothed_combined.columns,
        'Mean Absolute SHAP Value': np.abs(shap_smoothed_combined).mean(axis=0)
    })
    importance_smoothed_df.sort_values('Mean Absolute SHAP Value', ascending=False, inplace=True)

    # Define output paths and subdirectories for smoothed combinations
    smoothed_shap_settings_path = join(shap_smoothed_combined_dir, 'shap_settings.csv')
    smoothed_feature_analysis_path = join(shap_smoothed_combined_dir, 'shap_feature_analysis.csv')
    smoothed_feature_importance_path = join(shap_smoothed_combined_dir, 'shap_feature_importance.csv')
    smoothed_summary_plot_top_path = join(shap_smoothed_combined_dir, 'shap_summary_plot_top')
    smoothed_summary_plot_full_path = join(shap_smoothed_combined_dir, 'shap_summary_plot_full')
    smoothed_feature_importance_top_path = join(shap_smoothed_combined_dir, 'shap_feature_importance_top')
    smoothed_feature_importance_full_path = join(shap_smoothed_combined_dir, 'shap_feature_importance_full')
    smoothed_feature_effect_plots_dir = join(shap_smoothed_combined_dir, 'feature_effect_plots')

    if generate_smoothed_effect_plots:
        os.makedirs(smoothed_feature_effect_plots_dir, exist_ok=True)

    # Save SHAP analysis settings for smoothed combinations
    if not exists(smoothed_shap_settings_path):
        settings_df = pd.DataFrame({
            'Setting': [
                'polynomial_degree',
                'remove_outliers',
                'plot_feature_percentile_range',
                'no_top_features'
            ],
            'Value': [
                poly_degree,
                remove_outliers,
                plot_feature_percentile_range,
                no_top_features
            ]
        })
        settings_df.to_csv(smoothed_shap_settings_path, index=False)
        print("Smoothed combinations: SHAP settings saved.")

    # Save feature importance for smoothed combinations
    importance_smoothed_df.to_csv(smoothed_feature_importance_path, index=False)

    # Calculate comprehensive feature statistics including neutral effect values
    if not exists(smoothed_feature_analysis_path):
        warnings.filterwarnings('ignore', message='Polyfit may be poorly conditioned')

        # Pre-extract data
        X_smoothed_values = X_smoothed_combined.values
        shap_smoothed_array = shap_smoothed_combined.values

        # Calculate relationship statistics and neutral effect values for each combined feature
        feature_stats = []
        for feature in X_smoothed_combined.columns:
            feature_idx = X_smoothed_combined.columns.get_loc(feature)
            x = X_smoothed_values[:, feature_idx]
            y_shap = shap_smoothed_array[:, feature_idx]

            # Normalize for stable fitting
            x_scale = np.std(x)
            if x_scale != 0:
                x_mean = np.mean(x)
                x_norm = (x - x_mean) / x_scale

                # Linear fit
                lin_coef = np.polyfit(x_norm, y_shap, 1)
                lin_pred = np.polyval(lin_coef, x_norm)
                ss_tot = np.sum((y_shap - np.mean(y_shap))**2)
                lin_r2 = 1 - (np.sum((y_shap - lin_pred)**2) / ss_tot) if ss_tot > 0 else 0.0

                # Polynomial fit
                poly_coef = np.polyfit(x_norm, y_shap, poly_degree)
                poly_pred = np.polyval(poly_coef, x_norm)
                poly_r2 = 1 - (np.sum((y_shap - poly_pred)**2) / ss_tot) if ss_tot > 0 else 0.0

                # Determine predominant direction based on percentage of positive effects
                pct_positive = (y_shap > 0).sum() / len(y_shap) * 100
                if pct_positive > 70:
                    direction = "Positive"
                elif pct_positive < 30:
                    direction = "Negative"
                else:
                    direction = "Mixed"

                # Calculate neutral effect value using adaptive quantile-based binning
                # Neutral effect value: feature value where average SHAP impact approaches zero
                # Useful for setting covariate features to fixed neutral values during spatial predictions
                # e.g. when creating raster outputs where certain features must be held constant
                # Binning approach averages SHAP values within local neighbourhoods to find robust neutral point
                # Alternative single-sample approach unreliable due to outliers and feature interactions
                # Adaptive bin count scales with sample size (sqrt of n) to balance resolution and reliability
                # Quantile-based bins ensure equal sample distribution across bins regardless of feature distribution
                # Prevents sparse bins in tail regions that would produce unreliable estimates
                n_bins = int(np.sqrt(len(x)))
                bin_edges = np.percentile(x, np.linspace(0, 100, n_bins + 1))
                bin_indices = np.digitize(x, bin_edges)

                bin_means = []
                bin_shaps = []
                bin_counts = []
                for i in range(1, len(bin_edges)):
                    mask = bin_indices == i
                    count = mask.sum()
                    if count > 0:
                        bin_means.append(x[mask].mean())
                        bin_shaps.append(y_shap[mask].mean())
                        bin_counts.append(count)

                # Find bin where mean SHAP closest to zero
                if len(bin_shaps) > 0:
                    neutral_idx = np.argmin(np.abs(bin_shaps))
                    neutral_value = bin_means[neutral_idx]
                    neutral_shap = bin_shaps[neutral_idx]
                else:
                    neutral_value = np.median(x)
                    neutral_shap = 0.0
            else:
                # Zero variance feature
                x_mean = np.mean(x)
                lin_coef = np.array([0, 0])
                poly_coef = np.zeros(poly_degree + 1)
                lin_r2 = 0
                poly_r2 = 0
                direction = "Mixed"
                neutral_value = x_mean
                neutral_shap = 0.0
                n_bins = 0

            feature_stats.append({
                'Feature': feature,
                'Feature_Mean': x_mean,
                'Feature_Std': x_scale,
                'Linear_Coefficient': lin_coef[0],
                'Linear_R2': lin_r2,
                'Polynomial_Coefficients': ','.join(map(str, poly_coef)),
                'Polynomial_R2': poly_r2,
                'Predominant_Direction': direction,
                'Neutral_Effect_Value': neutral_value,
                'SHAP_at_Neutral': neutral_shap,
                'Neutral_Effect_Bins': n_bins
            })

        # Save feature statistics sorted alphabetically by feature name
        stats_df = pd.DataFrame(feature_stats)
        stats_df.sort_values('Feature', inplace=True)
        stats_df.to_csv(smoothed_feature_analysis_path, index=False)
        print("Smoothed combinations: feature relationship statistics and neutral effect values generated.")
    else:
        print("Smoothed combinations: feature statistics already exist.")

    # Convert to arrays for plotting
    shap_values_smoothed_array = shap_smoothed_combined.values

    # Generate importance plots for smoothed combinations
    if generate_smoothed_importance_plots:
        if not exists(f"{smoothed_feature_importance_top_path}.png") or not exists(f"{smoothed_feature_importance_top_path}.svg"):
            shap.summary_plot(shap_values_smoothed_array, X_smoothed_combined, plot_type="bar", show=False, max_display=no_top_features)
            plt.gcf().set_size_inches(12, 0.4 * no_top_features)
            plt.tight_layout()
            if not exists(f"{smoothed_feature_importance_top_path}.png"):
                plt.savefig(f"{smoothed_feature_importance_top_path}.png", bbox_inches='tight', dpi=150, pad_inches=0.1)
            if not exists(f"{smoothed_feature_importance_top_path}.svg"):
                plt.savefig(f"{smoothed_feature_importance_top_path}.svg", bbox_inches='tight', dpi=150, pad_inches=0.1, format='svg')
            plt.close()
            print("Smoothed combinations: top feature importance plot created.")

        if not exists(f"{smoothed_feature_importance_full_path}.png") or not exists(f"{smoothed_feature_importance_full_path}.svg"):
            shap.summary_plot(shap_values_smoothed_array, X_smoothed_combined, plot_type="bar", show=False, max_display=len(X_smoothed_combined.columns))
            plt.gcf().set_size_inches(12, 0.4 * len(X_smoothed_combined.columns))
            plt.tight_layout()
            if not exists(f"{smoothed_feature_importance_full_path}.png"):
                plt.savefig(f"{smoothed_feature_importance_full_path}.png", bbox_inches='tight', dpi=150, pad_inches=0.1)
            if not exists(f"{smoothed_feature_importance_full_path}.svg"):
                plt.savefig(f"{smoothed_feature_importance_full_path}.svg", bbox_inches='tight', dpi=150, pad_inches=0.1, format='svg')
            plt.close()
            print("Smoothed combinations: full feature importance plot created.")

    # Generate summary plots for smoothed combinations
    if generate_smoothed_summary_plots:
        if not exists(f"{smoothed_summary_plot_top_path}.png") or not exists(f"{smoothed_summary_plot_top_path}.svg"):
            shap.summary_plot(shap_values_smoothed_array, X_smoothed_combined, show=False, max_display=no_top_features)
            plt.gcf().set_size_inches(12, 0.4 * no_top_features)
            plt.tight_layout()
            if not exists(f"{smoothed_summary_plot_top_path}.png"):
                plt.savefig(f"{smoothed_summary_plot_top_path}.png", bbox_inches='tight', dpi=150, pad_inches=0.1)
            if not exists(f"{smoothed_summary_plot_top_path}.svg"):
                plt.savefig(f"{smoothed_summary_plot_top_path}.svg", bbox_inches='tight', dpi=150, pad_inches=0.1, format='svg')
            plt.close()
            print("Smoothed combinations: top summary plot created.")
        if generate_smoothed_summary_full_plot:
            if not exists(f"{smoothed_summary_plot_full_path}.png") or not exists(f"{smoothed_summary_plot_full_path}.svg"):
                shap.summary_plot(shap_values_smoothed_array, X_smoothed_combined, show=False, max_display=len(X_smoothed_combined.columns))
                plt.gcf().set_size_inches(12, 0.4 * len(X_smoothed_combined.columns))
                plt.tight_layout()
                if not exists(f"{smoothed_summary_plot_full_path}.png"):
                    plt.savefig(f"{smoothed_summary_plot_full_path}.png", bbox_inches='tight', dpi=150, pad_inches=0.1)
                if not exists(f"{smoothed_summary_plot_full_path}.svg"):
                    plt.savefig(f"{smoothed_summary_plot_full_path}.svg", bbox_inches='tight', dpi=150, pad_inches=0.1, format='svg')
                plt.close()
                print("Smoothed combinations: full summary plot created.")

    # Generate feature effect plots using statistics from feature analysis CSV
    if generate_smoothed_effect_plots:
        # Load feature statistics containing coefficients and neutral effect values
        feature_stats_df = pd.read_csv(smoothed_feature_analysis_path)

        # Pre-extract data for plotting
        X_smoothed_values = X_smoothed_combined.values
        shap_smoothed_array = shap_smoothed_combined.values
        feature_names_smoothed = importance_smoothed_df['Feature'].tolist()

        for feature in feature_names_smoothed:
            feature_effect_png_path = join(smoothed_feature_effect_plots_dir, f'feature_effect_{re.sub(r'[<>:"/\\|?*]', '_', feature)}.png')
            feature_effect_svg_path = join(smoothed_feature_effect_plots_dir, f'feature_effect_{re.sub(r'[<>:"/\\|?*]', '_', feature)}.svg')
            if not exists(feature_effect_png_path) or not exists(feature_effect_svg_path):
                # Get feature data
                feature_idx = X_smoothed_combined.columns.get_loc(feature)
                feature_values = X_smoothed_values[:, feature_idx]
                shap_values_feature = shap_smoothed_array[:, feature_idx]

                # Get feature statistics for trend lines and neutral effect value from alphabetically sorted CSV
                feature_row = feature_stats_df[feature_stats_df['Feature'] == feature].iloc[0]
                x_mean = feature_row['Feature_Mean']
                x_scale = feature_row['Feature_Std']
                neutral_value = feature_row['Neutral_Effect_Value']

                # Parse polynomial coefficients from CSV
                poly_coef = np.array([float(c) for c in feature_row['Polynomial_Coefficients'].split(',')])
                lin_coef = np.array([feature_row['Linear_Coefficient'], 0])

                # Determine x range for plotting with optional outlier removal for visualisation
                if remove_outliers:
                    lower = np.percentile(feature_values, (100 - plot_feature_percentile_range) / 2)
                    upper = np.percentile(feature_values, 100 - (100 - plot_feature_percentile_range) / 2)
                    mask = (feature_values >= lower) & (feature_values <= upper)
                    feature_values_plot = feature_values[mask]
                    shap_values_plot = shap_values_feature[mask]
                    x_range = np.linspace(lower, upper, 200)
                else:
                    feature_values_plot = feature_values
                    shap_values_plot = shap_values_feature
                    x_range = np.linspace(feature_values.min(), feature_values.max(), 200)

                fig, ax = plt.subplots(figsize=(10, 6))
                ax.scatter(feature_values_plot, shap_values_plot, alpha=0.5, rasterized=True)

                # Draw trend lines using coefficients from statistics CSV
                if x_scale != 0:
                    x_range_norm = (x_range - x_mean) / x_scale

                    # Linear trend
                    lin_pred = np.polyval(lin_coef, x_range_norm)
                    ax.plot(x_range, lin_pred, "--", color='black', alpha=0.8, label='Linear trend')

                    # Polynomial trend
                    poly_pred = np.polyval(poly_coef, x_range_norm)
                    ax.plot(x_range, poly_pred, "--", color='red', alpha=0.8, label=f'Polynomial trend (degree {poly_degree})')

                # Add neutral effect line from statistics CSV
                ax.axvline(x=neutral_value, color='blue', linestyle=':', linewidth=2, alpha=0.8, label='Neutral effect value')

                ax.set_title(f'SHAP Feature Effect: {feature}')
                ax.set_xlabel(f'{feature} value')
                ax.set_ylabel('SHAP value (impact on prediction)')
                ax.legend(loc='upper right')
                if not exists(feature_effect_png_path):
                    plt.savefig(feature_effect_png_path, bbox_inches='tight', dpi=150, pad_inches=0.1)
                if not exists(feature_effect_svg_path):
                    plt.savefig(feature_effect_svg_path, bbox_inches='tight', dpi=150, pad_inches=0.1, format='svg')
                plt.close()

        print("Smoothed combinations: feature effect plots generated.")

if generate_conceptual_combinations:
    print("Processing conceptual feature combinations...")

    for combo_idx, conceptual_combinations in enumerate(conceptual_combinations_list, 1):
        print(f"\nProcessing conceptual combination set {combo_idx}...")

        # Create directory for this combination set
        conceptual_dir = join(model_dataset_dir, f'shap_conceptual_combination_{combo_idx}')
        os.makedirs(conceptual_dir, exist_ok=True)

        # Save grouping rules as CSV
        grouping_rules_data = []
        for concept_name, patterns in conceptual_combinations.items():
            for pattern in patterns:
                grouping_rules_data.append({'Concept': concept_name, 'Pattern': pattern})

        grouping_rules_df = pd.DataFrame(grouping_rules_data)
        grouping_rules_path = join(conceptual_dir, 'conceptual_grouping_rules.csv')
        grouping_rules_df.to_csv(grouping_rules_path, index=False)

        # Create conceptual combinations by summing SHAP values of related features
        shap_conceptual_combined = pd.DataFrame()

        for concept_name, patterns in conceptual_combinations.items():
            # Find original features matching patterns, then map to renamed features
            matching_original_features = []
            for pattern in patterns:
                matching_original_features.extend([orig_name for orig_name in feature_name_mapping.keys() if pattern in orig_name])

            # Remove duplicates and convert to renamed features
            matching_original_features = list(dict.fromkeys(matching_original_features))
            matching_renamed_features = [feature_name_mapping[orig] for orig in matching_original_features if feature_name_mapping[orig] in X_clean.columns]

            if matching_renamed_features:
                # Sum SHAP values for all features in this conceptual group
                concept_shap_sum = shap_clean[matching_renamed_features].sum(axis=1)
                shap_conceptual_combined[concept_name] = concept_shap_sum
                print(f"  '{concept_name}': {len(matching_renamed_features)} features combined")

        # Calculate importance for conceptual combinations
        importance_conceptual_df = pd.DataFrame({
            'Feature': shap_conceptual_combined.columns,
            'Mean Absolute SHAP Value': np.abs(shap_conceptual_combined).mean(axis=0)
        })
        importance_conceptual_df.sort_values('Mean Absolute SHAP Value', ascending=False, inplace=True)

        # Define output paths for this conceptual combination
        conceptual_feature_importance_path = join(conceptual_dir, 'shap_feature_importance.csv')
        conceptual_feature_importance_plot_path = join(conceptual_dir, 'shap_feature_importance')

        # Save feature importance
        importance_conceptual_df.to_csv(conceptual_feature_importance_path, index=False)

        # Convert to array for plotting
        shap_values_conceptual_array = shap_conceptual_combined.values

        # Create dummy feature values dataframe for plotting compatibility
        X_conceptual_dummy = pd.DataFrame(np.zeros_like(shap_values_conceptual_array),
                                         columns=shap_conceptual_combined.columns)

        # Generate importance plot for this conceptual combination
        if not exists(f"{conceptual_feature_importance_plot_path}.png") or not exists(f"{conceptual_feature_importance_plot_path}.svg"):
            shap.summary_plot(shap_values_conceptual_array, X_conceptual_dummy, plot_type="bar", show=False)
            plt.gcf().set_size_inches(12, 0.4 * len(shap_conceptual_combined.columns))
            plt.tight_layout()
            if not exists(f"{conceptual_feature_importance_plot_path}.png"):
                plt.savefig(f"{conceptual_feature_importance_plot_path}.png", bbox_inches='tight', dpi=150, pad_inches=0.1)
            if not exists(f"{conceptual_feature_importance_plot_path}.svg"):
                plt.savefig(f"{conceptual_feature_importance_plot_path}.svg", bbox_inches='tight', dpi=150, pad_inches=0.1, format='svg')
            plt.close()
            print(f"  Conceptual combination {combo_idx}: feature importance plot created.")

print("Feature combination analysis complete. All combined plots and statistics generated.")

# Compare accuracy (GEDI AGBD)

In [ ]:
# Imports for this section
import geopandas as gpd
import requests
from concurrent.futures import ThreadPoolExecutor
from osgeo import gdal

# Directories
gedi_raster_final_dir = join(base_dir, "2_targets/gedi_raster_final")
masks_dir = join(base_dir, "1_areas/masks")
polygons_dir = join(base_dir, "1_areas/polygons")
accuracy_dir = join(model_dataset_dir, "accuracy_comparison")
gedi_baseline_dir = join(accuracy_dir, "gedi_baseline_data")
makedirs(gedi_baseline_dir, exist_ok=True)

# Global function: Sample raster values
def sample_raster_values(pd_dataframe, raster_path, geom_x, geom_y, feature=False, n_threads=1):
    # Derive column name from filename
    raster_name = raster_path.split('/')[-1][:-4]
    if feature: raster_name = 'fea_' + raster_name
    # Load raster and extract metadata
    raster = gdal.Open(raster_path)
    band = raster.GetRasterBand(1)
    geotransform = raster.GetGeoTransform()
    raster_array = band.ReadAsArray()
    nodata = band.GetNoDataValue()
    rows, cols = raster_array.shape
    fill_value = nodata if nodata is not None else np.nan
    # Initialise output array with nodata
    sampled_values = np.full(len(geom_x), fill_value, dtype=raster_array.dtype)
    # Worker function for threaded sampling
    def sample_chunk(start, end):
        x_idx = ((geom_x[start:end] - geotransform[0]) / geotransform[1]).astype(int)
        y_idx = ((geom_y[start:end] - geotransform[3]) / geotransform[5]).astype(int)
        valid = (x_idx >= 0) & (x_idx < cols) & (y_idx >= 0) & (y_idx < rows)
        local_values = np.full(end - start, fill_value, dtype=raster_array.dtype)
        local_values[valid] = raster_array[y_idx[valid], x_idx[valid]]
        sampled_values[start:end] = local_values
    # Split points into chunks and process in parallel
    n_points = len(geom_x)
    chunk_size = (n_points + n_threads - 1) // n_threads
    chunk_ranges = [(i, min(i + chunk_size, n_points)) for i in range(0, n_points, chunk_size)]
    with ThreadPoolExecutor(max_workers=n_threads) as executor:
        executor.map(lambda r: sample_chunk(*r), chunk_ranges)
    # Assign to dataframe and release resources
    pd_dataframe[raster_name] = sampled_values
    raster = band = None

nodatavalue = -11111

In [ ]:
# Select target variate (uncomment one, must match model target)

# Use L4D naming convention
selected_variate = "agbd"
# selected_variate = "cover_z_000"
# selected_variate = "rh_010"
# selected_variate = "rh_020"
# selected_variate = "rh_030"
# selected_variate = "rh_040"
# selected_variate = "rh_050"
# selected_variate = "rh_060"
# selected_variate = "rh_070"
# selected_variate = "rh_080"
# selected_variate = "rh_090"
# selected_variate = "rh_095"
# selected_variate = "rh_098"

l4d_gpkg_url = "https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4D_Imputed_Waveforms/comp/GEDI_L4D_20190418_20230316_validation_metrics.gpkg"

# Template for clipping
template_gdf = gpd.read_file(join(polygons_dir, "template.gpkg"))
template_bounds = template_gdf.total_bounds

In [ ]:
# Compile GEDI baseline data

# Template for clipping
template_gdf = gpd.read_file(join(polygons_dir, "template.gpkg"))
template_bounds = template_gdf.total_bounds

# Determine if L4B is needed (AGBD only)
need_l4b = selected_variate == "agbd"

# Search for L4D rasters (original UTM projections)
gedi04d_final_dir = join(gedi_raster_final_dir, "GEDI04_D")
l4d_rasters = []
if exists(gedi04d_final_dir):
    for f in os.listdir(gedi04d_final_dir):
        if f.startswith("GEDI04_D_original_") and f.endswith(f"_{selected_variate}.tif"):
            l4d_rasters.append(join(gedi04d_final_dir, f))
    if l4d_rasters: print(f"L4D: found {len(l4d_rasters)} raster(s) for '{selected_variate}'")
    else: print(f"L4D: no rasters found for '{selected_variate}'")
else: print("L4D: directory not found")

# Search for L4B raster (AGBD only, MU variant)
gedi04b_final_dir = join(gedi_raster_final_dir, "GEDI04_B")
l4b_raster = None
if need_l4b:
    if exists(gedi04b_final_dir):
        for f in os.listdir(gedi04b_final_dir):
            if f.startswith("GEDI04_B_original_") and "MU" in f:
                l4b_raster = join(gedi04b_final_dir, f)
                print(f"L4B: found '{f}'")
                break
        if not l4b_raster: print("L4B: no MU raster found")
    else: print("L4B: directory not found")

# Check if required data exists
missing = []
if not l4d_rasters: missing.append("L4D")
if need_l4b and not l4b_raster: missing.append("L4B")
if missing:
    print(f"\nMissing data: {', '.join(missing)}. Run '2_variates.ipynb' to download GEDI rasters.")
else:
    print("\nAll required GEDI data found. Proceeding...")

    # Copy L4D rasters to baseline directory
    print("\nCopying L4D rasters")
    l4d_copied = []
    for src in l4d_rasters:
        dst = join(gedi_baseline_dir, os.path.basename(src))
        if not exists(dst):
            print(f"  Copying: {os.path.basename(src)}")
            gdal.Translate(dst, src, options=gdal.TranslateOptions(format='GTiff', creationOptions=['COMPRESS=ZSTD', 'ZSTD_LEVEL=1']))
        else: print(f"  Exists: {os.path.basename(src)}")
        l4d_copied.append(dst)

    # Copy L4B raster if needed
    l4b_copied = None
    if need_l4b and l4b_raster:
        print("\nCopying L4B raster")
        dst = join(gedi_baseline_dir, os.path.basename(l4b_raster))
        if not exists(dst):
            print(f"  Copying: {os.path.basename(l4b_raster)}")
            gdal.Translate(dst, l4b_raster, options=gdal.TranslateOptions(format='GTiff', creationOptions=['COMPRESS=ZSTD', 'ZSTD_LEVEL=1']))
        else: print(f"  Exists: {os.path.basename(l4b_raster)}")
        l4b_copied = dst

    # Create masked versions for presentation
    print("\nCreating masked rasters")
    mask_2023 = join(masks_dir, "mask_forest_2023.tif")
    mask_2021 = join(masks_dir, "mask_forest_2021.tif")

    # Mask raster to forest extent, resampling mask if needed.
    def apply_forest_mask(raster_path, mask_path, output_path):
        if exists(output_path):
            print(f"  Masked exists: {os.path.basename(output_path)}")
            return
        raster_ds = gdal.Open(raster_path)
        raster_proj = raster_ds.GetProjection()
        raster_gt = raster_ds.GetGeoTransform()
        raster_xsize, raster_ysize = raster_ds.RasterXSize, raster_ds.RasterYSize
        raster_bounds = [raster_gt[0], raster_gt[3] + raster_gt[5] * raster_ysize,
                         raster_gt[0] + raster_gt[1] * raster_xsize, raster_gt[3]]
        mask_resampled = gdal.Warp('', mask_path, options=gdal.WarpOptions(
            format='MEM', dstSRS=raster_proj, outputBounds=raster_bounds,
            width=raster_xsize, height=raster_ysize, resampleAlg='near', dstNodata=nodatavalue))
        mask_array = mask_resampled.GetRasterBand(1).ReadAsArray()
        raster_array = raster_ds.GetRasterBand(1).ReadAsArray()
        raster_array = np.where(mask_array == 1, raster_array, nodatavalue)
        driver = gdal.GetDriverByName('GTiff')
        out_ds = driver.Create(output_path, raster_xsize, raster_ysize, 1, raster_ds.GetRasterBand(1).DataType,
                               options=['COMPRESS=ZSTD', 'ZSTD_LEVEL=1'])
        out_ds.SetGeoTransform(raster_gt)
        out_ds.SetProjection(raster_proj)
        out_band = out_ds.GetRasterBand(1)
        out_band.WriteArray(raster_array)
        out_band.SetNoDataValue(nodatavalue)
        out_ds = raster_ds = mask_resampled = None
        print(f"  Created: {os.path.basename(output_path)}")

    # Mask L4D rasters (2023 forest)
    for raster_path in l4d_copied:
        masked_name = os.path.basename(raster_path).replace("_original_", "_masked_")
        apply_forest_mask(raster_path, mask_2023, join(gedi_baseline_dir, masked_name))

    # Mask L4B raster (2021 forest)
    if l4b_copied:
        masked_name = os.path.basename(l4b_copied).replace("_original_", "_masked_")
        apply_forest_mask(l4b_copied, mask_2021, join(gedi_baseline_dir, masked_name))

    # Download and clip L4D validation gpkg
    print("\nL4D validation metrics")
    gpkg_raw = join(gedi_baseline_dir, "GEDI_L4D_validation_metrics_raw.gpkg")
    gpkg_clipped = join(gedi_baseline_dir, "GEDI_L4D_validation_metrics_clipped.gpkg")

    if not exists(gpkg_raw):
        print("  Downloading L4D validation gpkg...")
        response = requests.get(l4d_gpkg_url, stream=True)
        response.raise_for_status()
        with open(gpkg_raw, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print("  Download complete")
    else: print("  Raw gpkg exists")

    if not exists(gpkg_clipped):
        print("  Clipping gpkg to template extent...")
        tiles_gdf = gpd.read_file(gpkg_raw)
        tiles_gdf = tiles_gdf.to_crs(template_gdf.crs)
        tiles_clipped = gpd.clip(tiles_gdf, template_gdf)
        tiles_clipped.to_file(gpkg_clipped, driver="GPKG")
        print(f"  Clipped to {len(tiles_clipped)} tiles")
    else: print("  Clipped gpkg exists")

In [ ]:
# Calculate weighted L4D accuracy metrics

# Weighting logic: each tile's contribution proportional to forest pixel count within it.
# This accounts for partial overlap and non-forest areas.
# RMSE aggregated via MSE-weighting (statistically valid for combining RMSEs).
# R² approximated using predicted variance as proxy for observed variance (valid for k=1 NN).

print("Calculating L4D accuracy")
tiles_gdf = gpd.read_file(gpkg_clipped)

# Determine column names based on variate
if selected_variate == "agbd": rmse_col, mean_col = "rmse_agbd", "agbd_mean_valid"
elif selected_variate == "cover_z_000": rmse_col, mean_col = "rmse_cover", "cover_mean_valid"
else:
    rh_num = selected_variate.replace("rh_", "").lstrip("0") or "0"
    rmse_col, mean_col = f"rmse_{rh_num}", f"rh_mean_valid_{rh_num}"

# Verify columns exist
if rmse_col not in tiles_gdf.columns or mean_col not in tiles_gdf.columns:
    raise ValueError(f"Columns '{rmse_col}' or '{mean_col}' not found in gpkg")

# Load reference L4D raster for pixel counting and variance calculation
ref_raster = l4d_copied[0]
ref_ds = gdal.Open(ref_raster)
ref_proj = ref_ds.GetProjection()
ref_gt = ref_ds.GetGeoTransform()
ref_xsize, ref_ysize = ref_ds.RasterXSize, ref_ds.RasterYSize
ref_bounds = [ref_gt[0], ref_gt[3] + ref_gt[5] * ref_ysize, ref_gt[0] + ref_gt[1] * ref_xsize, ref_gt[3]]

# Load L4D array for variance calculation
l4d_array = ref_ds.GetRasterBand(1).ReadAsArray().astype(float)
l4d_nodata = ref_ds.GetRasterBand(1).GetNoDataValue()
ref_ds = None
l4d_array[l4d_array == l4d_nodata] = np.nan

# Resample forest mask to reference grid
mask_aligned = gdal.Warp('', mask_2023, options=gdal.WarpOptions(
    format='MEM', dstSRS=ref_proj, outputBounds=ref_bounds,
    width=ref_xsize, height=ref_ysize, resampleAlg='near', dstNodata=nodatavalue))
mask_array = mask_aligned.GetRasterBand(1).ReadAsArray()
mask_aligned = None

# Reproject tiles to raster CRS
from osgeo import osr
raster_srs = osr.SpatialReference()
raster_srs.ImportFromWkt(ref_proj)
raster_epsg = raster_srs.GetAuthorityCode(None)
tiles_reproj = tiles_gdf.to_crs(epsg=int(raster_epsg))

# Calculate forest pixel counts and R² approximation per tile
forest_counts = []
r2_approx_list = []
for idx, tile in tiles_reproj.iterrows():
    minx, miny, maxx, maxy = tile.geometry.bounds
    col_start = max(0, int((minx - ref_gt[0]) / ref_gt[1]))
    col_end = min(ref_xsize, int((maxx - ref_gt[0]) / ref_gt[1]))
    row_start = max(0, int((ref_gt[3] - maxy) / abs(ref_gt[5])))
    row_end = min(ref_ysize, int((ref_gt[3] - miny) / abs(ref_gt[5])))

    if col_end > col_start and row_end > row_start:
        tile_mask = mask_array[row_start:row_end, col_start:col_end]
        forest_count = np.sum(tile_mask == 1)
        # Variance from forest pixels only
        tile_pixels = l4d_array[row_start:row_end, col_start:col_end]
        forest_pixels = tile_pixels[(tile_mask == 1) & (~np.isnan(tile_pixels))]
        if len(forest_pixels) > 1:
            var_pred = np.var(forest_pixels)
            rmse_val = tiles_gdf.loc[idx, rmse_col]
            r2 = 1 - (rmse_val**2 / var_pred) if var_pred > 0 else np.nan
        else: r2 = np.nan
    else:
        forest_count = 0
        r2 = np.nan

    forest_counts.append(forest_count)
    r2_approx_list.append(r2)

tiles_gdf['forest_pixels'] = forest_counts
tiles_gdf['r2_approx'] = r2_approx_list
tiles_gdf['rrmse'] = (tiles_gdf[rmse_col] / tiles_gdf[mean_col]) * 100

# Filter to tiles with forest data and valid metrics
tiles_valid = tiles_gdf[(tiles_gdf['forest_pixels'] > 0)].dropna(subset=['rrmse', 'r2_approx'])

# Weighted aggregation
weights = tiles_valid['forest_pixels'].values
weighted_rmse = np.sqrt(np.average(tiles_valid[rmse_col].values**2, weights=weights))
weighted_rrmse = np.average(tiles_valid['rrmse'].values, weights=weights)
weighted_r2 = np.average(tiles_valid['r2_approx'].values, weights=weights)

print(f"  Tiles with data: {len(tiles_valid)}")
print(f"  Total forest pixels: {weights.sum():,}")
print(f"  Weighted RMSE: {weighted_rmse:.4f}")
print(f"  Weighted rRMSE: {weighted_rrmse:.4f}%")
print(f"  Weighted R² (approx): {weighted_r2:.4f}")

# Save L4D tile-weighted accuracy
l4d_accuracy = {
    'source': 'L4D (tile-weighted)',
    'variate': selected_variate,
    'rmse': weighted_rmse,
    'rrmse': weighted_rrmse,
    'r2': weighted_r2,
    'n_tiles': len(tiles_valid),
    'n_forest_pixels': int(weights.sum())
}
l4d_accuracy_df = pd.DataFrame([l4d_accuracy])
l4d_accuracy_path = join(accuracy_dir, "l4d_tile_accuracy.csv")
l4d_accuracy_df.to_csv(l4d_accuracy_path, index=False)
print(f"  Saved: {l4d_accuracy_path}")

In [ ]:
# Print available final datasets

print("Available datasets")
for f in os.listdir(datasets_dir):
    if f.endswith(".pkl"):
        print(f'original_dataset = "{f}"')

In [ ]:
# Point-based accuracy comparison

# Paste the correct dataset from above
original_dataset = "agbd.pkl"

# Load original dataset to recover geometry and year
print("Loading datasets")
original_df = pd.read_pickle(join(datasets_dir, original_dataset))
model_df = model_dataset.copy()

# Join on shot number (model_df uses 'shot_number', original uses 'tar_shot_number')
print(f"  Model dataset: {len(model_df)} points")
original_subset = original_df[['tar_shot_number', 'tar_geometry', 'tar_year']].copy()
accuracy_df = model_df.merge(original_subset, left_on='shot_number', right_on='tar_shot_number', how='left')
accuracy_df = accuracy_df.dropna(subset=['tar_geometry'])
print(f"  After geometry join: {len(accuracy_df)} points")

# Split by year for sampling
l4d_years = [2024] # Year after
l4b_years = [2020, 2021, 2022]
accuracy_l4d = accuracy_df[accuracy_df['tar_year'].isin(l4d_years)].copy()
accuracy_l4b = accuracy_df[accuracy_df['tar_year'].isin(l4b_years)].copy() if need_l4b else None

print(f"  L4D-eligible points (2023): {len(accuracy_l4d)}")
if need_l4b: print(f"  L4B-eligible points (2019-2021): {len(accuracy_l4b)}")

# Sample L4D rasters (may be multiple UTM zones)
print("\nSampling L4D rasters")
l4d_sampled_col = f"l4d_{selected_variate}"
if len(accuracy_l4d) > 0:
    # Sample each zone raster, merge results (first non-nodata wins)
    l4d_samples = np.full(len(accuracy_l4d), nodatavalue, dtype=float)
    for raster_path in l4d_copied:
        print(f"  Sampling: {os.path.basename(raster_path)}")
        # Get raster CRS
        raster_ds = gdal.Open(raster_path)
        raster_proj = raster_ds.GetProjection()
        raster_ds = None
        srs = osr.SpatialReference()
        srs.ImportFromWkt(raster_proj)
        raster_epsg = int(srs.GetAuthorityCode(None))
        # Reproject points and extract coordinates
        gdf = gpd.GeoDataFrame(accuracy_l4d, geometry='tar_geometry', crs='EPSG:4326')
        gdf_reproj = gdf.to_crs(epsg=raster_epsg)
        geom_x = gdf_reproj.geometry.x.values
        geom_y = gdf_reproj.geometry.y.values
        # Sample raster
        temp_df = pd.DataFrame(index=accuracy_l4d.index)
        sample_raster_values(temp_df, raster_path, geom_x, geom_y)
        values = temp_df.iloc[:, 0].values
        # Fill where we don't have data yet
        mask = (l4d_samples == nodatavalue) & (values != nodatavalue)
        l4d_samples[mask] = values[mask]
    accuracy_l4d[l4d_sampled_col] = l4d_samples
    # Filter out nodata
    accuracy_l4d_valid = accuracy_l4d[accuracy_l4d[l4d_sampled_col] != nodatavalue].copy()
    print(f"  Valid samples: {len(accuracy_l4d_valid)} / {len(accuracy_l4d)}")
else:
    accuracy_l4d_valid = accuracy_l4d

# Sample L4B raster (AGBD only)
l4b_sampled_col = "l4b_agbd"
accuracy_l4b_valid = None
if need_l4b and l4b_copied and accuracy_l4b is not None and len(accuracy_l4b) > 0:
    print("\nSampling L4B raster")
    print(f"  Sampling: {os.path.basename(l4b_copied)}")
    # Get raster CRS
    raster_ds = gdal.Open(l4b_copied)
    raster_proj = raster_ds.GetProjection()
    raster_ds = None
    srs = osr.SpatialReference()
    srs.ImportFromWkt(raster_proj)
    raster_epsg = int(srs.GetAuthorityCode(None))
    # Reproject points and extract coordinates
    gdf = gpd.GeoDataFrame(accuracy_l4b, geometry='tar_geometry', crs='EPSG:4326')
    gdf_reproj = gdf.to_crs(epsg=raster_epsg)
    geom_x = gdf_reproj.geometry.x.values
    geom_y = gdf_reproj.geometry.y.values
    # Sample raster
    sample_raster_values(accuracy_l4b, l4b_copied, geom_x, geom_y)
    accuracy_l4b.rename(columns={accuracy_l4b.columns[-1]: l4b_sampled_col}, inplace=True)
    # Filter out nodata
    accuracy_l4b_valid = accuracy_l4b[accuracy_l4b[l4b_sampled_col] != nodatavalue].copy()
    print(f"  Valid samples: {len(accuracy_l4b_valid)} / {len(accuracy_l4b)}")

# Calculate accuracy metrics
print("\nCalculating point-based accuracy")
results = []

# L4D accuracy
if len(accuracy_l4d_valid) > 0:
    l4d_metrics = calculate_metrics(
        accuracy_l4d_valid[selected_target].values,
        accuracy_l4d_valid[l4d_sampled_col].values
    )
    l4d_metrics = {k: v for k, v in l4d_metrics.items() if k != 'optimal_value'}
    l4d_metrics['source'] = 'L4D (point)'
    l4d_metrics['n_points'] = len(accuracy_l4d_valid)
    results.append(l4d_metrics)
    print(f"  L4D: R²={l4d_metrics['r2']:.4f}, RMSE={l4d_metrics['rmse']:.4f}, rRMSE={l4d_metrics['rrmse']:.4f}%")

# L4B accuracy (AGBD only)
if need_l4b and accuracy_l4b_valid is not None and len(accuracy_l4b_valid) > 0:
    l4b_metrics = calculate_metrics(
        accuracy_l4b_valid[selected_target].values,
        accuracy_l4b_valid[l4b_sampled_col].values
    )
    l4b_metrics = {k: v for k, v in l4b_metrics.items() if k != 'optimal_value'}
    l4b_metrics['source'] = 'L4B (point)'
    l4b_metrics['n_points'] = len(accuracy_l4b_valid)
    results.append(l4b_metrics)
    print(f"  L4B: R²={l4b_metrics['r2']:.4f}, RMSE={l4b_metrics['rmse']:.4f}, rRMSE={l4b_metrics['rrmse']:.4f}%")

# Load model validation metrics from model_description
print("\nLoading model validation metrics")
with open(model_description_dir) as f:
    model_desc = json.load(f)

model_metrics = {
    'source': 'Model (CV)',
    'r2': model_desc.get('score_validation (r2) mean', np.nan),
    'me': model_desc.get('score_validation (me) mean', np.nan),
    'rmse': model_desc.get('score_validation (rmse) mean', np.nan),
    'rrmse': model_desc.get('score_validation (rrmse) mean', np.nan),
    'n_points': f"{n_splits - n_hpo_splits} folds"
}
results.append(model_metrics)
print(f"  Model: R²={model_metrics['r2']:.4f}, RMSE={model_metrics['rmse']:.4f}, rRMSE={model_metrics['rrmse']:.4f}%")

# Build comparison dataframe
comparison_df = pd.DataFrame(results)
comparison_df = comparison_df.set_index('source').T

# Determine best performer per metric
def best_source(row, metric_name):
    opt = optimal_values.get(metric_name, 'min')
    numeric_vals = {k: v for k, v in row.items() if isinstance(v, (int, float)) and not np.isnan(v)}
    if not numeric_vals: return '-'
    if opt == 'max': return max(numeric_vals, key=numeric_vals.get)
    return min(numeric_vals, key=numeric_vals.get)

comparison_df['best'] = comparison_df.apply(
    lambda row: best_source(row, row.name) if row.name in optimal_values else '-', axis=1
)

# Save comparison
comparison_path = join(accuracy_dir, "accuracy_comparison.csv")
comparison_df.to_csv(comparison_path)
print(f"\nComparison saved: {comparison_path}")
print(comparison_df.to_string())

# Disconnect runtime

In [ ]:
# Useful for stopping background execution
runtime.unassign()